# Grover's serach for RNA design

In [2]:
import numpy as np
from math import pi,log2,floor,ceil
from qiskit import *
from qiskit.circuit import *
from qiskit.extensions import *
from qiskit.circuit.library import *
from qiskit.extensions.simulator.snapshot import snapshot
from qiskit.quantum_info.operators import Operator
from qiskit.extensions.simulator.snapshot import snapshot
from scipy import optimize
from matplotlib.pyplot import plot,show
%matplotlib inline
%config InlineBackend.figure_format = 'svg' # Makes the images look nice

## 1 - Addition

Let's implement the circuit in [1] modified to add mod 2^n.

In [3]:
#MAJ
q=QuantumRegister(3)
MAJ = QuantumCircuit(q)

MAJ.cnot(q[2],q[1])
MAJ.cnot(q[2],q[0])
MAJ.ccx(q[0],q[1],q[2])

maj = MAJ.to_gate(label='MAJ')

#UMA
q=QuantumRegister(3)
UMA = QuantumCircuit(q)

UMA.ccx(q[0],q[1],q[2])
UMA.cnot(q[2],q[0])
UMA.cnot(q[0],q[1])

uma = UMA.to_gate(label='UMA')

#add
def add_circuit(nb_bits):
    concerved = QuantumRegister(nb_bits, name="=")
    not_concerved = QuantumRegister(nb_bits, name="->")
    initial_carry = QuantumRegister(1)
    add = QuantumCircuit(initial_carry,concerved,not_concerved)
    add.append(maj,[initial_carry,not_concerved[0],concerved[0]])
    for i in range(1,nb_bits):
        add.append(maj,[concerved[i-1],not_concerved[i],concerved[i]])
    for i in range(nb_bits-1,0,-1):
        add.append(uma,[concerved[i-1],not_concerved[i],concerved[i]])
    add.append(uma,[initial_carry,not_concerved[0],concerved[0]])
    return add

def add_gate(nb_bits):
    return add_circuit(nb_bits).to_gate(label="add{}".format(nb_bits))

add_circuit(6).draw()

┌──────┐                                                                »
  q2: ┤0     ├────────────────────────────────────────────────────────────────»
      │      │┌──────┐                                                        »
 =_0: ┤2     ├┤0     ├────────────────────────────────────────────────────────»
      │      ││      │┌──────┐                                                »
 =_1: ┤      ├┤2     ├┤0     ├────────────────────────────────────────────────»
      │      ││      ││      │┌──────┐                                ┌──────┐»
 =_2: ┤      ├┤      ├┤2     ├┤0     ├────────────────────────────────┤0     ├»
      │  MAJ ││      ││      ││      │┌──────┐                ┌──────┐│      │»
 =_3: ┤      ├┤      ├┤      ├┤2     ├┤0     ├────────────────┤0     ├┤2     ├»
      │      ││  MAJ ││      ││      ││      │┌──────┐┌──────┐│      ││      │»
 =_4: ┤      ├┤      ├┤      ├┤      ├┤2     ├┤0     ├┤0     ├┤2     ├┤      ├»
      │      ││      ││  MAJ ││      ││      ││      ││      ││      ││      │»
 =_5: ┤      ├┤      ├┤      ├┤      ├┤      ├┤2     ├┤2     ├┤      ├┤      ├»
      │      ││      ││      ││  MAJ ││      ││      ││      ││      ││  UMA │»
->_0: ┤1     ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├»
      └──────┘│      ││      ││      ││  MAJ ││      ││      ││  UMA ││      │»
->_1: ────────┤1     ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├»
              └──────┘│      ││      ││      ││  MAJ ││  UMA ││      ││      │»
->_2: ────────────────┤1     ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├»
                      └──────┘│      ││      ││      ││      ││      ││      │»
->_3: ────────────────────────┤1     ├┤      ├┤      ├┤      ├┤      ├┤1     ├»
                              └──────┘│      ││      ││      ││      │└──────┘»
->_4: ────────────────────────────────┤1     ├┤      ├┤      ├┤1     ├────────»
                                      └──────┘│      ││      │└──────┘        »
->_5: ────────────────────────────────────────┤1     ├┤1     ├────────────────»
                                              └──────┘└──────┘                »
«                      ┌──────┐
«  q2: ────────────────┤0     ├
«              ┌──────┐│      │
« =_0: ────────┤0     ├┤2     ├
«      ┌──────┐│      ││      │
« =_1: ┤0     ├┤2     ├┤      ├
«      │      ││      ││      │
« =_2: ┤2     ├┤      ├┤      ├
«      │      ││      ││  UMA │
« =_3: ┤      ├┤      ├┤      ├
«      │      ││  UMA ││      │
« =_4: ┤      ├┤      ├┤      ├
«      │  UMA ││      ││      │
« =_5: ┤      ├┤      ├┤      ├
«      │      ││      ││      │
«->_0: ┤      ├┤      ├┤1     ├
«      │      ││      │└──────┘
«->_1: ┤      ├┤1     ├────────
«      │      │└──────┘        
«->_2: ┤1     ├────────────────
«      └──────┘                
«->_3: ────────────────────────
«                              
«->_4: ────────────────────────
«                              
«->_5: ────────────────────────
«

And also a circuit to increment and a circuit to decrement.

In [4]:
def incr_circuit(nb_bits):
    number = QuantumRegister(nb_bits, name="number")
    incr = QuantumCircuit(number)
    for i in range(nb_bits-1):
        incr.append(MCXGate(nb_bits-i-1),[number[j] for j in range(nb_bits-i)])
    incr.x(number[0])
    return incr

def incr_gate(nb_bits):
    return incr_circuit(nb_bits).to_gate(label="incr{}".format(nb_bits))

def decr_circuit(nb_bits):
    number = QuantumRegister(nb_bits, name="number")
    decr = QuantumCircuit(number)
    for i in range(nb_bits-1):
        decr.append(MCXGate(nb_bits-i-1,ctrl_state="0"*(nb_bits-i-1)),[number[j] for j in range(nb_bits-i)])
    decr.x(number[0])
    return decr

def decr_gate(nb_bits):
    return decr_circuit(nb_bits).to_gate(label="decr{}".format(nb_bits))

incr_circuit(6).draw()
decr_circuit(6).draw()

┌───┐
number_0: ──o────o────o────o────o──┤ X ├
            │    │    │    │  ┌─┴─┐└───┘
number_1: ──o────o────o────o──┤ X ├─────
            │    │    │  ┌─┴─┐└───┘     
number_2: ──o────o────o──┤ X ├──────────
            │    │  ┌─┴─┐└───┘          
number_3: ──o────o──┤ X ├───────────────
            │  ┌─┴─┐└───┘               
number_4: ──o──┤ X ├────────────────────
          ┌─┴─┐└───┘                    
number_5: ┤ X ├─────────────────────────
          └───┘

## 2 - Matchnig parentesis

`(`, `.` and `)` are encoded respectively by 1, 0 and -1, which are encoded with tow's complement representation.

In [5]:
nb_bits_bracket_dot = 2
nb_bits_loop_type = 2

def n_to_ctrl_state(n,nb_bits):
    return format(n,"0{}b".format(nb_bits))[:nb_bits] 

#if test_concerved == tested_to_0 then tested_to_0 is set to 0 and will trigger the application of some gate in then_append(circuit,tested_to_0,nb_bits_test,gate,...)
def start_if_equal(circuit,tested_to_0,nb_bits_test,test_concerved):
    for i in range(nb_bits_test):
        circuit.cx(test_concerved[i],tested_to_0[i])
end_if_equal=start_if_equal

#if test_const == tested_to_0 then tested_to_0 is set to 0 and will trigger the application of some gate in then_append(circuit,tested_to_0,nb_bits_test,gate,...)
def start_if_equal_const(circuit,tested_to_0,nb_bits_test,test_const):
    for i,bit in enumerate(n_to_ctrl_state(test_const,nb_bits_test)):
        if "1" == bit:
            circuit.x(tested_to_0[i])

end_if_equal_const=start_if_equal_const

def then_append(circuit,test_0,nb_bits_test,gate,qbits_list):
    circuit.append(gate.control(nb_bits_test,ctrl_state="0"*nb_bits_test),[test_0[i] for i in range(nb_bits_test)]+qbits_list)
    return    
    
def matching_parenthesis_circuit(length,truncation_point=50):
    nb_bits_position=ceil(log2(length+1)) #length+1 because we add a ( at the begining to check the folding is well formed
    #add_positions=add_gate(nb_bits_position)
    incr_position=incr_gate(nb_bits_position)
    decr_position=decr_gate(nb_bits_position)
    #nb_bits_extend = max(0,nb_bits_position-nb_bits_bracket_dot)
    nb_bits_third_bit_loop_type = nb_bits_position
    
    #input
    #semantics of folding[0:nb_bits_bracket_dot]
    #. == "00"
    #h == "11"
    #) == "10"
    #( == "01"
    folding = QuantumRegister(nb_bits_bracket_dot,name="(/./h/)")
    
    #auxillary qbits
    k_folding = QuantumRegister(nb_bits_position,name="k_folding")
    count = QuantumRegister(nb_bits_position,name="count") #count how many parenthesis still need to be closed. folding[k_folding=0] is initiallised to ( and count[k_count=0] is used to check that there is no extra ) that would match in the folding
    k_count = QuantumRegister(nb_bits_position,name="k_count")
    continue_searching_right_matching_left = QuantumRegister(1,name="continue_match") #continue_searching_right_matching_left[k] corresponds to count[k]
    
    change_folding_encoding = QuantumRegister(1,name="change_encoding")
    continue_scanning = QuantumRegister(1,name="continue_scan")
    zero = QuantumRegister(1,name="zero")
    #one = QuantumRegister(1,name="one")
    
    #results
    k_right_matching_left = QuantumRegister(nb_bits_position,name="k_match") #k_right_matching_left[k_left] is initialised to k_left and is incremented until count[k_left]==0, that is until k_right_matching_left[k_left] is the indice of the ) matching the ( == folding[k_left]
    loop = QuantumRegister(nb_bits_position,name="loop") #length of the loop starting at folding[k_left]==(. loop[k_left] is initialised to 0 and is incremented each time count[k_left]==1 until count[k_left]==0
    
    #semantics of loop_type
    #where * is anything well matched of any lenght (including 0)
    #      + is anything well matched of length > 0
    #      - is any number of dots (including 0)
    #0 init phase (?       / scan phase ((*)?             / stack    ((*))
    #1                       scan phase (.-(*)?
    #2                       scan phase (.-?              / hairpin  (-)
    #3                       scan phase (.-(*).-?         / interior (.-(*).-)
    #4                       scan phase ((*)+?            / bulge    ((*)+)
    #5                       scan phase (.-(*)*(*)?
    #6                                                      bulge    (+(*))
    #7                       scan phase (.-(*)*(*).-?     / multi    (.-(*)*(*).-)
    
    #so the loop_type is coded on 3 bits,
    #but actually, the last bit is coded by loop_type[nb_bits_loop_type:nb_bits_loop_type+nb_bits_position] with the folowing semantics
    #loop_type[nb_bits_loop_type:nb_bits_loop_type+nb_bits_position] -> 3rd bit of loop_type
    #0                                                               -> 0
    #!= 0                                                            -> 1
    
    #transitions
    # -> transition to another state
    # <- transition to oneself
    #transition only if continue_searching_right_matching_left
    #when at a level >=2 just wait until beeing again at level 1
    
    
    #transitions grouped by start point
    
    #init phase only
    #0 (?            --[((]-> 0 ((*)?
    #0 (?            --[(.]-> 2 (.-?
    #0 (?            --[()]-> 2 (-)           hairpin
    
    #scan phase
    #0 ((*)?         --[))]-> 0 ((*))         stack
    #0 ((*)?         --[).]-> 4 ((*)+?
    #0 ((*)?         --[)(]-> 4 ((*)+?
    
    #1 (.-(*)?       --[).]-> 3 (.-(*).-?
    #1 (.-(*)?       --[))]-> 6 (+(*))        bulge
    #1 (.-(*)?       --[)(]-> 5 (.-(*)*(*)?
    
    #2 (.-?          <-[..]--
    #2 (.-?          --[.(]-> 1 (.-(*)?
    #2 (.-?          --[.)]-> 2 (-)           hairpin
    
    #3 (.-(*).-?     <-[..]--
    #3 (.-(*).-?     --[.)]-> 3 (.-(*).-)     interior
    #3 (.-(*).-?     --[.(]-> 5 (.-(*)*(*)?
    
    #once 3rd bit have been set to 1, it will neve be set to 0 again
    #incerment 3rd bit every time a transition to n>=4 occurs if tit is possible to transitionate from m<=3 to n
    
    #4 ((*)+?        --[*)]-> 4 ((*)+)        bulge
    #4 ((*)+?        <-[*.]--
    #4 ((*)+?        <-[*(]--
    
    #5 (.-(*)*(*)?   <-[)(]--
    #5 (.-(*)*(*)?   --[))]-> 6 (+(*))        bulge
    #5 (.-(*)*(*)?   --[).]-> 7 (.-(*)*(*).-?
    
    #7 (.-(*)*(*).-? <-[..]--
    #7 (.-(*)*(*).-? --[.(]-> 5 (.-(*)*(*)?
    #7 (.-(*)*(*).-? --[.)]-> 7 (.-(*)*(*).-) multi
    
    
    #transitions grouped by end point
    #0 (? init phase --[((]-> 0 ((*)?
    #0 ((*)?         --[))]-> 0 ((*))         stack
    
    #2 (.-?          --[.(]-> 1 (.-(*)?
    
    #0 (? init phase --[(.]-> 2 (.-?
    #0 (? init phase --[()]-> 2 (-)           hairpin
    #2 (.-?          --[.)]-> 2 (.-)          hairpin
    #2 (.-?          <-[..]--
    
    #1 (.-(*)?       --[).]-> 3 (.-(*).-?
    #3 (.-(*).-?     --[.)]-> 3 (.-(*).-)     interior
    #3 (.-(*).-?     <-[..]--
    
    #0 ((*)?         --[).]-> 4 ((*)+?                   INDIST-
    #4 ((*)+?        <-[*.]--                            -INGUISHABLE -> increment 3rd bit
    #0 ((*)?         --[)(]-> 4 ((*)+?                   INDIST-
    #4 ((*)+?        <-[*(]--                            -INGUISHABLE -> increment 3rd bit
    #4 ((*)+?        --[*)]-> 4 ((*)+)        bulge
    
    #3 (.-(*).-?     --[.(]-> 5 (.-(*)*(*)?             INDIST-
    #7 (.-(*)*(*).-? --[.(]-> 5 (.-(*)*(*)?             -INGUISHABLE -> increment 3rd bit
    #1 (.-(*)?       --[)(]-> 5 (.-(*)*(*)?             INDIST-
    #5 (.-(*)*(*)?   <-[)(]--                           -INGUISHABLE -> increment 3rd bit
    
    #1 (.-(*)?       --[))]-> 6 (+(*))        bulge      INDIST-
    #5 (.-(*)*(*)?   --[))]-> 6 (+(*))        bulge      -INGUISHABLE -> increment 3rd bit
    
    #5 (.-(*)*(*)?   --[).]-> 7 (.-(*)*(*).-?
    #7 (.-(*)*(*).-? --[.)]-> 7 (.-(*)*(*).-) multi
    #7 (.-(*)*(*).-? <-[..]--
    
    loop_type = QuantumRegister(nb_bits_loop_type,name="loop_type")
    third_bit_loop_type = QuantumRegister(nb_bits_third_bit_loop_type,name="loop_type[{}]_encoding)".format(nb_bits_loop_type))
    incorrect_folding = QuantumRegister(nb_bits_position,name="incorrect")
    correct_folding = QuantumRegister(1,name="correct")
    
    #circuit = QuantumCircuit(folding,k_folding,change_folding_encoding,k_count,count,k_right_matching_left,zero,continue_searching_right_matching_left,continue_scanning,loop,incorrect_folding,correct_folding) #order of registers as they appear along the program
    circuit = QuantumCircuit()
    
    def semantics(bit_string):
        i_bit = 0
        semantics_string = ""
        
        def semantics_number(label,nb_bits_number):
            s=str(int(bit_string[i_bit:i_bit+nb_bits_number],2))
            delta=nb_bits_number-len(s)
            return s+"_"+label+delta*" "+" ",nb_bits_number
        
        if circuit.has_register(correct_folding):
            s,i=semantics_number("?",1)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(incorrect_folding):
            s,i=semantics_number("!",nb_bits_position)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(loop_type):
            s,i=semantics_number("lt",nb_bits_loop_type)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(loop):
            s,i=semantics_number("lo",nb_bits_position)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(continue_scanning):
            s,i=semantics_number("sc",1)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(continue_searching_right_matching_left):
            s,i=semantics_number("se",1)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(zero):
            semantics_string+="cst"
            semantics_string+=bit_string[i_bit:i_bit+1]
            i_bit+=1
        #if circuit.has_register(one):
        #    semantics_string+=bit_string[i_bit:i_bit+1]
        #    i_bit+=1
            semantics_string+=" "
        if circuit.has_register(k_right_matching_left):
            s,i=semantics_number("km",nb_bits_position)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(count):
            s,i=semantics_number("c",nb_bits_position)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(k_count):
            s,i=semantics_number("kc",nb_bits_position)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(change_folding_encoding):
            semantics_string+="-" if bit_string[i_bit:i_bit+1]=="0" else "h"
            i_bit+=1
            semantics_string+=" "
        if circuit.has_register(k_folding):
            s,i=semantics_number("kf",nb_bits_position)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(folding):
            semantics_string+=("." if bit_string[i_bit:i_bit+nb_bits_bracket_dot]=="00" else
                               "h" if bit_string[i_bit:i_bit+nb_bits_bracket_dot]=="11" else
                               "(" if bit_string[i_bit:i_bit+nb_bits_bracket_dot]=="01" else
                               ")" if bit_string[i_bit:i_bit+nb_bits_bracket_dot]=="10" else "")
            i_bit+=nb_bits_bracket_dot
        return semantics_string

    
    
    #1) Superposition at the begining of the Grover search
    
    #any symbol
    circuit.add_register(folding)
    for i in range(nb_bits_bracket_dot):
        circuit.h(folding[i])
    
    if  truncation_point == "any folding" or truncation_point <= 0:
        return circuit,semantics
    
    #at any position
    circuit.add_register(k_folding)
    circuit.h(k_folding)
    
    if truncation_point == "any k_folding" or truncation_point <= 1:
        return circuit,semantics
    
    #except at position n_to_ctrl_state(0,nb_bits_position), where it is necessarily a ( == "01" to check if the folding is well formed
    for i in range(nb_bits_bracket_dot):
        circuit.append(HGate().control(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[k_folding[j] for j in range(nb_bits_position)]+[folding[i]])
    circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[k_folding[j] for j in range(nb_bits_position)]+[folding[0]])
    
    #except at positions k > length, where folding[k] is set to .
    for k in range(length+1,2**nb_bits_position):
        for i in range(nb_bits_bracket_dot):
            circuit.append(HGate().control(nb_bits_position,ctrl_state=n_to_ctrl_state(k,nb_bits_position)),[k_folding[j] for j in range(nb_bits_position)]+[folding[i]])
    
    if truncation_point == "candidate folding" or truncation_point <= 2:
        return circuit,semantics
    
    circuit.add_register(change_folding_encoding)
    
    #if this symbol is h == "11", then change it to be . == "00"
    circuit.ccx(*[folding[i] for i in range(nb_bits_bracket_dot)],change_folding_encoding)
    circuit.append(MCMT(XGate(),1,2),[change_folding_encoding]+[folding[i] for i in range(nb_bits_bracket_dot)])
    
    if truncation_point == "encoding" or truncation_point <= 3:
        return circuit,semantics
    
    
    #2) Initialisation of the oracle
    
    #initialise k_count to k_left
    circuit.add_register(k_count)
    circuit.h(k_count)
    
    if truncation_point == "init k_count" or truncation_point <= 4:
        return circuit,semantics
    
    #if k_folding == k_count
    start_if_equal(circuit,k_count,nb_bits_position,k_folding)
    
    #initialise count with folding
    circuit.add_register(count)
    #folding -> count
    #. == "00" -> 0 == "0...000"
    #do nothing
    #) == "10" -> 0 == "0...000"
    #do nothing
    #( == "01" -> 1 == "0...001"
    then_append(circuit,k_count,nb_bits_position,CXGate(),[folding[0],count[0]])
    
    #end if k_folding == k_count
    end_if_equal(circuit,k_count,nb_bits_position,k_folding)
    
    if truncation_point == "init count" or truncation_point <= 5:
        return circuit,semantics
    
    #initialise k_right_matching_left[k_count] to k_left (and it will then be incremented up to the matchning k_right)
    circuit.add_register(k_right_matching_left)
    for i in range(nb_bits_position):
        circuit.cnot(k_count[i],k_right_matching_left[i])
    
    if truncation_point == "init k_right_matching_left" or truncation_point <= 6:
        return circuit,semantics
    
    #initialise one to 1
    #circuit.add_register(one)
    circuit.add_register(zero)
    #circuit.x(one)
    
    if truncation_point == "init cst" or truncation_point <= 7:
        return circuit,semantics
    
    #initialise continue_searching_right_matching_left=1 when 1 == count
    #indeed, when folding[k_left] == ( (which will be checked by there <*>) then there is initially 1 parenthesis to be closed
    circuit.add_register(continue_searching_right_matching_left)
    circuit.cnot(count[0],continue_searching_right_matching_left)
    
    if truncation_point == "init continue_searching_right_matching_left" or truncation_point <= 8:
        return circuit,semantics
    
    #initialise continue_scanning=1
    circuit.add_register(continue_scanning)
    circuit.x(continue_scanning)
    
    if truncation_point == "init continue_scanning" or truncation_point <= 9:
        return circuit,semantics
    
    circuit.add_register(loop)
    circuit.add_register(loop_type)
    circuit.add_register(third_bit_loop_type)
    #ancillary qbit for computing loop_type
    #it is cleaned every time after use
    #zero serves as should_transitionate
    should_transitionate=zero
    #should_transitionate serves as should_transitionate
    could_transitionate=should_transitionate
    
    #when needing to control by 0 != third_bit_loop_type
    #get an ancillary qbit loop_type[nb_bits_loop_type] and encode it by third_bit_loop_type like this
    #circuit.append(MCXGate(nb_bits_position,n_to_ctrl_state(0,nb_bits_position)),[third_bit_loop_type[i] for i in range(nb_bits_third_bit_loop_type)]+[loop_type[nb_bits_loop_type]])
    #but actually it never happens
    #in case it happens one day, remember to clean it before modifying third_bit_loop_type
    
    
    #3) Compute parameters by scanning the folding
    
    for k in range(length):
        
        if truncation_point <= 10 + k:
            return circuit,semantics
        
        #<*> if it has been checked that there is a matching ) to find, then continue_searching_right_matching_left[k_left]==1, otherwise continue_searching_right_matching_left[k_left]==0
        
        
        #3.1) k_right_matching_left
        
        #increment k_right_matching_left[k_left] if continue_searching_right_matching_left[k_left]==1
        circuit.append(incr_position.control(2),[continue_scanning,continue_searching_right_matching_left]+[k_right_matching_left[i] for i in range(nb_bits_position)])
        
        
        #3.2) continue_scanning
        
        #go to the next position
        #increment k_count
        circuit.append(incr_position,[k_count[i] for i in range(nb_bits_position)])
        
        #if the end has been reached, that is if k_count == 0
        #then do not continue_scanning
        circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[k_count[i] for i in range(nb_bits_position)]+[continue_scanning])
        
        
        #3.3) Transitions of loop_type
        
        #transitions grouped by end point
        
        #5 (.-(*)*(*)?   --[).]-> 7 (.-(*)*(*).-?
        #7 (.-(*)*(*).-? --[.)]-> 7 (.-(*)*(*).-) multi
        #7 (.-(*)*(*).-? <-[..]--
        
        #1 (.-(*)?       --[))]-> 6 (+(*))        bulge      INDIST-
        #5 (.-(*)*(*)?   --[))]-> 6 (+(*))        bulge      -INGUISHABLE -> increment 3rd bit
        
        #3 (.-(*).-?     --[.(]-> 5 (.-(*)*(*)?             INDIST-
        #7 (.-(*)*(*).-? --[.(]-> 5 (.-(*)*(*)?             -INGUISHABLE -> increment 3rd bit
        #1 (.-(*)?       --[)(]-> 5 (.-(*)*(*)?             INDIST-
        #5 (.-(*)*(*)?   <-[)(]--                           -INGUISHABLE -> increment 3rd bit
        
        #0 ((*)?         --[).]-> 4 ((*)+?                   INDIST-
        #4 ((*)+?        <-[*.]--                            -INGUISHABLE -> increment 3rd bit
        #0 ((*)?         --[)(]-> 4 ((*)+?                   INDIST-
        #4 ((*)+?        <-[*(]--                            -INGUISHABLE -> increment 3rd bit
        #4 ((*)+?        --[*)]-> 4 ((*)+)        bulge
        
        #1 (.-(*)?       --[).]-> 3 (.-(*).-?
        #3 (.-(*).-?     --[.)]-> 3 (.-(*).-)     interior
        #3 (.-(*).-?     <-[..]--
        
        #2 (.-?          --[.(]-> 1 (.-(*)?
        
        #0 (? init phase --[(.]-> 2 (.-?
        #0 (? init phase --[()]-> 2 (-)           hairpin
        #2 (.-?          --[.)]-> 2 (.-)          hairpin
        #2 (.-?          <-[..]--
        
        #0 (? init phase --[((]-> 0 ((*)?
        #0 ((*)?         --[))]-> 0 ((*))         stack
        
        
        #if k_folding == k_count
        start_if_equal(circuit,k_count,nb_bits_position,k_folding)
        
        if 0 == k:
            #3.3.1) init phase - the only possible state is 0
            #3.3.1.1) Transitions to loop_type=2
            #since 0 == k
            #then 0 == loop_type
            #so there is no need to control by loop_type
            
            #0 (? init phase --[(.]-> 2 (.-?
            #0 (? init phase --[()]-> 2 (-)           hairpin
            then_append(circuit,k_count,nb_bits_position,MCXGate(2+nb_bits_position+(nb_bits_bracket_dot-1),ctrl_state="0"+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[folding[0],loop_type[nb_bits_loop_type-1]])
            
            #3.3.1.2) Transitions to loop_type=0
            #0 (? init phase --[((]-> 0 ((*)?
            #do nothing
            
        #TODO in order to optimise
        #elif 1 == k:
        #    #3.3.2) possible states are 0,2
        #    #TODO
        #    pass
        #elif 2 == k:
        #    #3.3.3) possible states are 0,2,4,1
        #    #TODO
        #    pass
        #elif 3 == k:
        #    #3.3.4) possible states are 0,2,4,1,3,5,6
        #    #TODO
        #    pass
        else:
            #3.3.5) all states are possible
            #3.3.5.1) Transitions to loop_type=7 and loop_type=3
            #3.3.5.1.1) Detecting who should_transitionate to 7 or 3
            
            #1 (.-(*)?       --[).]-> 3 (.-(*).-?
            #5 (.-(*)*(*)?   --[).]-> 7 (.-(*)*(*).-?
            then_append(circuit,k_count,nb_bits_position,MCXGate(2+nb_bits_position+nb_bits_loop_type+nb_bits_bracket_dot,ctrl_state="00"+n_to_ctrl_state(1,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[folding[i] for i in range(nb_bits_bracket_dot)]+[should_transitionate])
            
            #3 (.-(*).-?     --[.)]-> 3 (.-(*).-)     interior
            #3 (.-(*).-?     <-[..]--
            #7 (.-(*)*(*).-? --[.)]-> 7 (.-(*)*(*).-) multi
            #7 (.-(*)*(*).-? <-[..]--
            #folding[k_count] == ) or . is characterised by 0 == folding[k_count][0]
            then_append(circuit,k_count,nb_bits_position,MCXGate(2+nb_bits_position+nb_bits_loop_type+1,ctrl_state="0"+n_to_ctrl_state(3,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[folding[0]]+[should_transitionate])
            
            
            #3.3.5.1.2) Transitionate by setting who should_transitionate to loop_type=7 ou loop_type=3
            
            #end if k_folding == k_count
            end_if_equal(circuit,k_count,nb_bits_position,k_folding)
            
            #decrement k_count
            circuit.append(decr_position,[k_count[i] for i in range(nb_bits_position)])
            
            #if k_folding == k_count
            start_if_equal(circuit,k_count,nb_bits_position,k_folding)
            
            #1 (.-(*)?       --[).]-> 3 (.-(*).-?
            #5 (.-(*)*(*)?   --[).]-> 7 (.-(*)*(*).-?
            start="".join(reversed(n_to_ctrl_state(1,nb_bits_loop_type))) #ctrl_state are in the wrong way
            end  ="".join(reversed(n_to_ctrl_state(3,nb_bits_loop_type))) #ctrl_state are in the wrong way
            for i in range(nb_bits_loop_type): #third_bit_loop_type is treated separately (and here there is nothing to do)
                if start[i] != end[i]:
                    #this transition is characterised by [)*], that is by 1 == folding[k_count][nb_bits_bracket_dot-1]
                    then_append(circuit,k_count,nb_bits_position,CCXGate(),[should_transitionate,folding[nb_bits_bracket_dot-1],loop_type[i]])
            
            #3 (.-(*).-?     --[.)]-> 3 (.-(*).-)     interior
            #3 (.-(*).-?     <-[..]--
            #7 (.-(*)*(*).-? --[.)]-> 7 (.-(*)*(*).-) multi
            #7 (.-(*)*(*).-? <-[..]--
            #do nothing
            
            #clean should_transitionate
            circuit.append(MCXGate(2+nb_bits_position+nb_bits_loop_type,ctrl_state=n_to_ctrl_state(3,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[should_transitionate])
            
            #end if k_folding == k_count
            end_if_equal(circuit,k_count,nb_bits_position,k_folding)
            
            #increment k_count
            circuit.append(incr_position,[k_count[i] for i in range(nb_bits_position)])
            
            
            
            
            
            #3.3.5.2) Transitions to loop_type=5
            #3.3.5.2.1) Detecting who should_transitionate to 5
            
            #if k_folding == k_count
            start_if_equal(circuit,k_count,nb_bits_position,k_folding)
            
            #3 (.-(*).-?     --[.(]-> 5 (.-(*)*(*)?             INDIST-
            #7 (.-(*)*(*).-? --[.(]-> 5 (.-(*)*(*)?             -INGUISHABLE -> increment 3rd bit
            #1 (.-(*)?       --[)(]-> 5 (.-(*)*(*)?             INDIST-
            #5 (.-(*)*(*)?   <-[)(]--                           -INGUISHABLE -> increment 3rd bit
            #and
            #7 (.-(*)*(*).-? --[.(]-> 5 (.-(*)*(*)?
            #should be differenciated from
            #5 (.-(*)*(*)?   --[).]-> 7 (.-(*)*(*).-?
            #7 (.-(*)*(*).-? --[.)]-> 7 (.-(*)*(*).-) multi
            #7 (.-(*)*(*).-? <-[..]--
            
            #folding[k_count] == ( is characterised by 1 == folding[k_count][0]
            #3 or 7 or 1 or 5 is characterised by 1 == loop_type[0]
            then_append(circuit,k_count,nb_bits_position,MCXGate(2+nb_bits_position+(nb_bits_loop_type-1)+(nb_bits_bracket_dot-1),ctrl_state="1"+n_to_ctrl_state(1,nb_bits_loop_type-1)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type-1)]+[folding[0],should_transitionate])
            
            
            #3.3.5.2.2) Transitionate by setting who should_transitionate to loop_type=5
            
            #end if k_folding == k_count
            end_if_equal(circuit,k_count,nb_bits_position,k_folding)
            
            #decrement k_count
            circuit.append(decr_position,[k_count[i] for i in range(nb_bits_position)])
            
            #if k_folding == k_count
            start_if_equal(circuit,k_count,nb_bits_position,k_folding)
            
            #3 (.-(*).-?     --[.(]-> 5 (.-(*)*(*)?             INDIST-
            #7 (.-(*)*(*).-? --[.(]-> 5 (.-(*)*(*)?             -INGUISHABLE -> increment 3rd bit
            start="".join(reversed(n_to_ctrl_state(3,nb_bits_loop_type)))
            end  ="".join(reversed(n_to_ctrl_state(5,nb_bits_loop_type)))
            for i in range(nb_bits_loop_type): #third_bit_loop_type is treated separately (and here there is nothing to do)
                if start[i] != end[i]:
                    #this transition is characterised by [.*] (against [)*]) so by 0 == folding[k_count][nb_bits_bracket_dot-1]
                    then_append(circuit,k_count,nb_bits_position,CCXGate(ctrl_state="01"),[should_transitionate,folding[nb_bits_bracket_dot-1],loop_type[i]])
            #(and increment third_bit_loop_type at the end)
            
            #1 (.-(*)?       --[)(]-> 5 (.-(*)*(*)?             INDIST-
            #5 (.-(*)*(*)?   <-[)(]--                           -INGUISHABLE -> increment 3rd bit
            #do nothing (except increment third_bit_loop_type at the end)
            
            #increment third_bit_loop_type
            circuit.append(incr_position.control(),[should_transitionate]+[third_bit_loop_type[i] for i in range(nb_bits_third_bit_loop_type)])
            
            #clean should_transitionate
            circuit.append(MCXGate(2+nb_bits_position+nb_bits_loop_type,ctrl_state=n_to_ctrl_state(5,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[should_transitionate])
            
            #end if k_folding == k_count
            end_if_equal(circuit,k_count,nb_bits_position,k_folding)
            
            #increment k_count
            circuit.append(incr_position,[k_count[i] for i in range(nb_bits_position)])
            
            
            
            
            
            #3.3.5.3) Transitions to loop_type=6 and loop_type=1
            #3.3.5.3.1) Detecting who should_transitionate to 6
            
            #if k_folding == k_count
            start_if_equal(circuit,k_count,nb_bits_position,k_folding)
            
            #2 (.-?          --[.(]-> 1 (.-(*)?
            #there is no ned to differenciate 2 from 6 because once 6 is reached, 0 == continue_searching_right_matching_left
            then_append(circuit,k_count,nb_bits_position,MCXGate(2+nb_bits_position+nb_bits_loop_type+(nb_bits_bracket_dot-1),ctrl_state="1"+n_to_ctrl_state(2,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[folding[nb_bits_bracket_dot-1],should_transitionate])
            
            #1 (.-(*)?       --[))]-> 6 (+(*))        bulge      INDIST-
            #5 (.-(*)*(*)?   --[))]-> 6 (+(*))        bulge      -INGUISHABLE -> increment 3rd bit
            #folding[k_count] == ) is characterised by 1 == folding[k_count][1]
            #1 or 5 is characterised by 0 == loop_type[1] and 1 == loop_type[0]
            then_append(circuit,k_count,nb_bits_position,MCXGate(2+nb_bits_position+nb_bits_loop_type+(nb_bits_bracket_dot-1),ctrl_state="1"+n_to_ctrl_state(1,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[folding[0],should_transitionate])
            
            
            #3.3.5.3.2) Transitionate by setting who should_transitionate to loop_type=6
            
            #should do the same thing whatever the transition (to 6 or to 1), so there is no need to control by folding[k_folding-1]
            
            #2 (.-?          --[.(]-> 1 (.-(*)?
            #1 (.-(*)?       --[))]-> 6 (+(*))        bulge      INDIST-
            #5 (.-(*)*(*)?   --[))]-> 6 (+(*))        bulge      -INGUISHABLE -> increment 3rd bit
            start="".join(reversed(n_to_ctrl_state(5,nb_bits_loop_type))) #ctrl_state are in the wrong way
            end  ="".join(reversed(n_to_ctrl_state(6,nb_bits_loop_type))) #ctrl_state are in the wrong way
            for i in range(nb_bits_loop_type): #third_bit_loop_type is treated separately
                if start[i] != end[i]:
                    circuit.cx(should_transitionate,loop_type[i])
            
            #increment third_bit_loop_type if transitionate to 6
            #that is if should transitionate and ) == folding[k_count]
            circuit.append(incr_position.control(2),[should_transitionate,folding[nb_bits_bracket_dot-1]]+[third_bit_loop_type[i] for i in range(nb_bits_third_bit_loop_type)])
            
            #note that the transition 1 --[*(]-> has aleady been treated, to transitionning to 1 won't pause any problem
            
            #clean should_transitionate
            circuit.append(MCXGate(2+nb_bits_position+nb_bits_loop_type,ctrl_state=n_to_ctrl_state(6,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[should_transitionate])
            
            
            
            
            
            #3.3.5.4) Transitions to loop_type=4
            
            #0 ((*)?         --[).]-> 4 ((*)+?                   INDIST-
            #4 ((*)+?        <-[*.]--                            -INGUISHABLE -> increment 3rd bit
            #0 ((*)?         --[)(]-> 4 ((*)+?                   INDIST-
            #4 ((*)+?        <-[*(]--                            -INGUISHABLE -> increment 3rd bit
            #4 ((*)+?        --[*)]-> 4 ((*)+)        bulge
            #use an ancillary qbit could_transitionate to detect the case 0 --[*)]-> 4 (the only case that  is not in the list above)
            circuit.x(could_transitionate)
            then_append(circuit,k_count,nb_bits_position,MCXGate((nb_bits_loop_type+nb_bits_third_bit_loop_type)+1,ctrl_state="1"+n_to_ctrl_state(0,nb_bits_loop_type+nb_bits_third_bit_loop_type)),[loop_type[i] for i in range(nb_bits_loop_type)]+[third_bit_loop_type[i] for i in range(nb_bits_third_bit_loop_type)]+[folding[nb_bits_bracket_dot-1],could_transitionate])
            
            #everyone verifying 0 == loop_type[0] and 1 == loop_type[1] transitionates (that is increment third_bit_loop_type) as far as they could tansitionate
            circuit.append(incr_position.control(2+nb_bits_position+nb_bits_loop_type+1,ctrl_state="1"+n_to_ctrl_state(0,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[could_transitionate]+[third_bit_loop_type[i] for i in range(nb_bits_third_bit_loop_type)])
            
            #clean could_transitionate
            #no more need to control by ) == folding[k_count] since the only 0 == loop_type that stay verify it
            #TO VERIFY : it seems to me that there is a need of then_append() here, even though there is no control by folding[k_count], but I am not sure
            then_append(circuit,k_count,nb_bits_position,MCXGate(nb_bits_loop_type+nb_bits_third_bit_loop_type,ctrl_state=n_to_ctrl_state(0,nb_bits_loop_type+nb_bits_third_bit_loop_type)),[loop_type[i] for i in range(nb_bits_loop_type)]+[third_bit_loop_type[i] for i in range(nb_bits_third_bit_loop_type)]+[could_transitionate])
            circuit.x(could_transitionate)
            
            
            
            
            
            #3.3.5.5) Transitions to loop_type=2 and loop_type=0
            #2 (.-?          --[.)]-> 2 (.-)          hairpin
            #2 (.-?          <-[..]--
            #0 ((*)?         --[))]-> 0 ((*))         stack
            #do nothing
        
        #3.4) Update count[k_left]
        
        #if continue_searching_right_matching_left[k_left]==1
        #then update count[k_left] for the next candidate
        if 0 == k:
            #necessarily if folding[k_left]==( then continue_searching_right_matching_left[k_left]==1
            #so it is ok not controlling by continue_searching_right_matching_left here
            #( is characterised by 1 == folding[k_count][0]
            then_append(circuit,k_count,nb_bits_position,incr_position.control(2,ctrl_state="11"),[continue_scanning,folding[0                    ]]+[count[i] for i in range(nb_bits_position)])
            #) is characterised by 1 == folding[nb_bits_bracket_dot-1]
            then_append(circuit,k_count,nb_bits_position,decr_position.control(2,ctrl_state="11"),[continue_scanning,folding[nb_bits_bracket_dot-1]]+[count[i] for i in range(nb_bits_position)])
        else:
            #( is characterised by 1 == folding[k_count][0]
            then_append(circuit,k_count,nb_bits_position,incr_position.control(3,ctrl_state="111"),[continue_scanning,continue_searching_right_matching_left,folding[0                    ]]+[count[i] for i in range(nb_bits_position)])
            #) is characterised by 1 == folding[nb_bits_bracket_dot-1]
            #so there is no need controling by folding[0]
            then_append(circuit,k_count,nb_bits_position,decr_position.control(3,ctrl_state="111"),[continue_scanning,continue_searching_right_matching_left,folding[nb_bits_bracket_dot-1]]+[count[i] for i in range(nb_bits_position)])
        
        #if continue_searching_right_matching_left[k_left]==0, then simply increment count[k_left] so that it never equals 0 again
        #it is ok not controlling by continue_scanning here
        if 0 != k:
            #as stated previously, if 0==k then necessarily if folding[k_left]==( then continue_searching_right_matching_left[k_left]==1
            #so there is no need doing this if 0==k
            then_append(circuit,k_count,nb_bits_position,incr_position.control(ctrl_state="0"),[continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)])
        
        #if 1 == count[k_left]
        start_if_equal_const(circuit,count,nb_bits_position,1)
        
        
        #3.5) Update loop[k_left]
        
        #if 1 == continue_searching_right_matching_left[k_left] then increment loop[k_left]
        if 0==k:
            #since the value 0==loop[k_left] is known then there is no need to incr_position in order to increment
            #a CXGate() is sufficient
            then_append(circuit,[k_count[i] for i in range(nb_bits_position)]+[count[i] for i in range(nb_bits_position)],2*nb_bits_position,CCXGate(),[continue_scanning,continue_searching_right_matching_left,loop[0]])
        else:
            then_append(circuit,[k_count[i] for i in range(nb_bits_position)]+[count[i] for i in range(nb_bits_position)],2*nb_bits_position,incr_position.control(2),[continue_scanning,continue_searching_right_matching_left]+[loop[i] for i in range(nb_bits_position)])
        
        #end if 1 == count[k_left]
        end_if_equal_const(circuit,count,nb_bits_position,1)
        
        #end if k_folding == k_count
        end_if_equal(circuit,k_count,nb_bits_position,k_folding)
        
        
        #3.6) Update continue_searching_right_matching_left
        
        #if count[k_left] == 0
        #then there is no matching ) to find
        circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[count[i] for i in range(nb_bits_position)]+[continue_searching_right_matching_left])
    
    if truncation_point == "compute matching" or truncation_point <= 10:
        return circuit,semantics
    
    #set k_count to its initial value by adding it 2**nb_bits_position - length
    for i,bit in enumerate(n_to_ctrl_state(2**nb_bits_position - length,nb_bits_position)):
        if '1'==bit:
            circuit.append(incr_gate(i+1),[k_count[j] for j in range(nb_bits_position-i-1,nb_bits_position)])

    if truncation_point == "restore k_count" or truncation_point <= 10 + length:
        return circuit,semantics    
    
    #Computed parameters
    #folding[kleft] -> continue_searching_right_matching_left[k_left] * k_right_matching_left[k_left] * loop                                 * k_count[k_left] * count[k_left]
    #(              -> 0 if a matching ) has been found               * k_right_matching_left         * number of . (at level 1) in the loop * k_left          * unspecified
    #(              -> 1 if a matching ) has not been found           * !=k_left                      * unspecified                          * k_left          * unspecified
    #. or )         -> 0                                              * k_left                        * 0                                    * k_left          * unspecified
    
    
    #4) Aggregate results
    
    #if each ( of the folding has a matching ) and if an extra ( at the beginning does not have a matching )
    #that is, if none of the conditions of incorrect_folding is fulfilled
    #then it is a correct_folding
    circuit.add_register(incorrect_folding)
    
    #checking if there is an exceeding )
    #there is if 0 == continue_searching_right_matching_left[k_left=0]
    #since the value 0==incorrect_folding is known then there is no need to add in order to increment
    #a CXGate() is sufficient
    circuit.append(MCXGate(nb_bits_position+1,ctrl_state="0"+n_to_ctrl_state(0,nb_bits_position)),[k_count[i] for i in range(nb_bits_position)]+[continue_searching_right_matching_left,incorrect_folding[0]])
    
    if truncation_point == "check incorrect )" or truncation_point <= 11 + length:
        return circuit,semantics
    
    #sum all the incorrect flags continue_searching_right_matching_left (1 per position) into incorrect_folding
    #the sum cannot overflow since if the flag continue_searching_right_matching_left[k_left=0] == 0 (at k_left=0 incorrect beeing 0) then there is a ) somewhere
    #so continue_searching_right_matching_left[k_left=somewhere] == 0
    #so incorrect_folding is not incremented at each position
    for k in range(1,length+1):
        circuit.append(incr_position.control(nb_bits_position+1,ctrl_state="1"+n_to_ctrl_state(k,nb_bits_position)),[k_count[i] for i in range(nb_bits_position)]+[continue_searching_right_matching_left]+[incorrect_folding[i] for i in range(nb_bits_position)])
    
    if truncation_point == "check incorrect (" or truncation_point <= 12 + length:
        return circuit,semantics
    
    #it is a correct_folding if 0==incorrect_folding
    circuit.add_register(correct_folding)
    circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[incorrect_folding[i] for i in range(nb_bits_position)]+[correct_folding])
    
    return circuit,semantics

def matching_parenthesis_gate(length,truncation_point=50):
    circ,_ = matching_parenthesis_circuit(length,truncation_point)
    return circ.to_gate(label="matching")

circ,_=matching_parenthesis_circuit(2)

circ.draw()

┌───┐┌───┐     ┌───┐┌───┐                    »
               (/./h/)_0: ┤ H ├┤ H ├─────┤ X ├┤ H ├───────■────────────»
                          ├───┤└─┬─┘┌───┐└─┬─┘└─┬─┘┌───┐  │            »
               (/./h/)_1: ┤ H ├──┼──┤ H ├──┼────┼──┤ H ├──■────────────»
                          ├───┤  │  └─┬─┘  │    │  └─┬─┘  │            »
             k_folding_0: ┤ H ├──o────o────o────■────■────┼────■───────»
                          ├───┤  │    │    │    │    │    │    │       »
             k_folding_1: ┤ H ├──o────o────o────■────■────┼────┼────■──»
                          └───┘                         ┌─┴─┐  │    │  »
         change_encoding: ──────────────────────────────┤ X ├──┼────┼──»
                          ┌───┐                         └───┘┌─┴─┐  │  »
               k_count_0: ┤ H ├──────────────────────────────┤ X ├──┼──»
                          ├───┤                              └───┘┌─┴─┐»
               k_count_1: ┤ H ├───────────────────────────────────┤ X ├»
                          └───┘                                   └───┘»
                 count_0: ─────────────────────────────────────────────»
                                                                       »
                 count_1: ─────────────────────────────────────────────»
                                                                       »
               k_match_0: ─────────────────────────────────────────────»
                                                                       »
               k_match_1: ─────────────────────────────────────────────»
                                                                       »
                    zero: ─────────────────────────────────────────────»
                                                                       »
          continue_match: ─────────────────────────────────────────────»
                          ┌───┐                                        »
           continue_scan: ┤ X ├────────────────────────────────────────»
                          └───┘                                        »
                  loop_0: ─────────────────────────────────────────────»
                                                                       »
                  loop_1: ─────────────────────────────────────────────»
                                                                       »
             loop_type_0: ─────────────────────────────────────────────»
                                                                       »
             loop_type_1: ─────────────────────────────────────────────»
                                                                       »
loop_type[2]_encoding)_0: ─────────────────────────────────────────────»
                                                                       »
loop_type[2]_encoding)_1: ─────────────────────────────────────────────»
                                                                       »
             incorrect_0: ─────────────────────────────────────────────»
                                                                       »
             incorrect_1: ─────────────────────────────────────────────»
                                                                       »
                 correct: ─────────────────────────────────────────────»
                                                                       »
«                          ┌───────┐                                        »
«               (/./h/)_0: ┤1      ├──■─────────────────────────────────────»
«                          │       │  │                                     »
«               (/./h/)_1: ┤2      ├──┼─────────────────────────────────────»
«                          │       │  │                                     »
«             k_folding_0: ┤  mcmt ├──┼────■────────────────────────────────»
«                          │       │  │    │                                »
«             k_folding_1: ┤       ├──┼────┼────■

statistics

In [6]:
print("width {} ; depth {}".format(circ.width(),circ.depth()))

width 23 ; depth 61


try to optimise for simulations by making a `step_gate` instead of explicit loop (does not work)

In [7]:
nb_bits_bracket_dot = 2
nb_bits_loop_type = 2

def n_to_ctrl_state(n,nb_bits):
    return format(n,"0{}b".format(nb_bits))[:nb_bits] 

#if test_concerved == tested_to_0 then tested_to_0 is set to 0 and will trigger the application of some gate in then_append(circuit,tested_to_0,nb_bits_test,gate,...)
def start_if_equal(circuit,tested_to_0,nb_bits_test,test_concerved):
    for i in range(nb_bits_test):
        circuit.cx(test_concerved[i],tested_to_0[i])
end_if_equal=start_if_equal

#if test_const == tested_to_0 then tested_to_0 is set to 0 and will trigger the application of some gate in then_append(circuit,tested_to_0,nb_bits_test,gate,...)
def start_if_equal_const(circuit,tested_to_0,nb_bits_test,test_const):
    for i,bit in enumerate(n_to_ctrl_state(test_const,nb_bits_test)):
        if "1" == bit:
            circuit.x(tested_to_0[i])

end_if_equal_const=start_if_equal_const

def then_append(circuit,test_0,nb_bits_test,gate,qbits_list):
    circuit.append(gate.control(nb_bits_test,ctrl_state="0"*nb_bits_test),[test_0[i] for i in range(nb_bits_test)]+qbits_list)
    return

def step_circuit(nb_bits_position,incr_position,decr_position,nb_bits_third_bit_loop_type):
    folding = QuantumRegister(nb_bits_bracket_dot,name="(/./h/)")
    
    #auxillary qbits
    k_folding = QuantumRegister(nb_bits_position,name="k_folding")
    count = QuantumRegister(nb_bits_position,name="count") #count how many parenthesis still need to be closed. folding[k_folding=0] is initiallised to ( and count[k_count=0] is used to check that there is no extra ) that would match in the folding
    k_count = QuantumRegister(nb_bits_position,name="k_count")
    continue_searching_right_matching_left = QuantumRegister(1,name="continue_match") #continue_searching_right_matching_left[k] corresponds to count[k]
    
    change_folding_encoding = QuantumRegister(1,name="change_encoding")
    continue_scanning = QuantumRegister(1,name="continue_scan")
    zero = QuantumRegister(1,name="zero")
    #one = QuantumRegister(1,name="one")
    
    #results
    k_right_matching_left = QuantumRegister(nb_bits_position,name="k_match") #k_right_matching_left[k_left] is initialised to k_left and is incremented until count[k_left]==0, that is until k_right_matching_left[k_left] is the indice of the ) matching the ( == folding[k_left]
    loop = QuantumRegister(nb_bits_position,name="loop") #length of the loop starting at folding[k_left]==(. loop[k_left] is initialised to 0 and is incremented each time count[k_left]==1 until count[k_left]==0
    
    #semantics of loop_type
    #where * is anything well matched of any lenght (including 0)
    #      + is anything well matched of length > 0
    #      - is any number of dots (including 0)
    #0 init phase (?       / scan phase ((*)?             / stack    ((*))
    #1                       scan phase (.-(*)?
    #2                       scan phase (.-?              / hairpin  (-)
    #3                       scan phase (.-(*).-?         / interior (.-(*).-)
    #4                       scan phase ((*)+?            / bulge    ((*)+)
    #5                       scan phase (.-(*)*(*)?
    #6                                                      bulge    (+(*))
    #7                       scan phase (.-(*)*(*).-?     / multi    (.-(*)*(*).-)
    
    #so the loop_type is coded on 3 bits,
    #but actually, the last bit is coded by loop_type[nb_bits_loop_type:nb_bits_loop_type+nb_bits_position] with the folowing semantics
    #loop_type[nb_bits_loop_type:nb_bits_loop_type+nb_bits_position] -> 3rd bit of loop_type
    #0                                                               -> 0
    #!= 0                                                            -> 1
    
    #transitions
    # -> transition to another state
    # <- transition to oneself
    #transition only if continue_searching_right_matching_left
    #when at a level >=2 just wait until beeing again at level 1
    
    
    #transitions grouped by start point
    
    #init phase only
    #0 (?            --[((]-> 0 ((*)?
    #0 (?            --[(.]-> 2 (.-?
    #0 (?            --[()]-> 2 (-)           hairpin
    
    #scan phase
    #0 ((*)?         --[))]-> 0 ((*))         stack
    #0 ((*)?         --[).]-> 4 ((*)+?
    #0 ((*)?         --[)(]-> 4 ((*)+?
    
    #1 (.-(*)?       --[).]-> 3 (.-(*).-?
    #1 (.-(*)?       --[))]-> 6 (+(*))        bulge
    #1 (.-(*)?       --[)(]-> 5 (.-(*)*(*)?
    
    #2 (.-?          <-[..]--
    #2 (.-?          --[.(]-> 1 (.-(*)?
    #2 (.-?          --[.)]-> 2 (-)           hairpin
    
    #3 (.-(*).-?     <-[..]--
    #3 (.-(*).-?     --[.)]-> 3 (.-(*).-)     interior
    #3 (.-(*).-?     --[.(]-> 5 (.-(*)*(*)?
    
    #once 3rd bit have been set to 1, it will neve be set to 0 again
    #incerment 3rd bit every time a transition to n>=4 occurs if tit is possible to transitionate from m<=3 to n
    
    #4 ((*)+?        --[*)]-> 4 ((*)+)        bulge
    #4 ((*)+?        <-[*.]--
    #4 ((*)+?        <-[*(]--
    
    #5 (.-(*)*(*)?   <-[)(]--
    #5 (.-(*)*(*)?   --[))]-> 6 (+(*))        bulge
    #5 (.-(*)*(*)?   --[).]-> 7 (.-(*)*(*).-?
    
    #7 (.-(*)*(*).-? <-[..]--
    #7 (.-(*)*(*).-? --[.(]-> 5 (.-(*)*(*)?
    #7 (.-(*)*(*).-? --[.)]-> 7 (.-(*)*(*).-) multi
    
    
    #transitions grouped by end point
    #0 (? init phase --[((]-> 0 ((*)?
    #0 ((*)?         --[))]-> 0 ((*))         stack
    
    #2 (.-?          --[.(]-> 1 (.-(*)?
    
    #0 (? init phase --[(.]-> 2 (.-?
    #0 (? init phase --[()]-> 2 (-)           hairpin
    #2 (.-?          --[.)]-> 2 (.-)          hairpin
    #2 (.-?          <-[..]--
    
    #1 (.-(*)?       --[).]-> 3 (.-(*).-?
    #3 (.-(*).-?     --[.)]-> 3 (.-(*).-)     interior
    #3 (.-(*).-?     <-[..]--
    
    #0 ((*)?         --[).]-> 4 ((*)+?                   INDIST-
    #4 ((*)+?        <-[*.]--                            -INGUISHABLE -> increment 3rd bit
    #0 ((*)?         --[)(]-> 4 ((*)+?                   INDIST-
    #4 ((*)+?        <-[*(]--                            -INGUISHABLE -> increment 3rd bit
    #4 ((*)+?        --[*)]-> 4 ((*)+)        bulge
    
    #3 (.-(*).-?     --[.(]-> 5 (.-(*)*(*)?             INDIST-
    #7 (.-(*)*(*).-? --[.(]-> 5 (.-(*)*(*)?             -INGUISHABLE -> increment 3rd bit
    #1 (.-(*)?       --[)(]-> 5 (.-(*)*(*)?             INDIST-
    #5 (.-(*)*(*)?   <-[)(]--                           -INGUISHABLE -> increment 3rd bit
    
    #1 (.-(*)?       --[))]-> 6 (+(*))        bulge      INDIST-
    #5 (.-(*)*(*)?   --[))]-> 6 (+(*))        bulge      -INGUISHABLE -> increment 3rd bit
    
    #5 (.-(*)*(*)?   --[).]-> 7 (.-(*)*(*).-?
    #7 (.-(*)*(*).-? --[.)]-> 7 (.-(*)*(*).-) multi
    #7 (.-(*)*(*).-? <-[..]--
    
    loop_type = QuantumRegister(nb_bits_loop_type,name="loop_type")
    third_bit_loop_type = QuantumRegister(nb_bits_third_bit_loop_type,name="loop_type[{}]_encoding)".format(nb_bits_loop_type))
    
    circuit = QuantumCircuit(folding,k_folding,change_folding_encoding,k_count,count,k_right_matching_left,zero,continue_searching_right_matching_left,continue_scanning,loop,loop_type,third_bit_loop_type) #order of registers as they appear along the program
    
    #ancillary qbit for computing loop_type
    #it is cleaned every time after use
    #zero serves as should_transitionate
    should_transitionate=zero
    #should_transitionate serves as should_transitionate
    could_transitionate=should_transitionate
    
    k=1
    #<*> if it has been checked that there is a matching ) to find, then continue_searching_right_matching_left[k_left]==1, otherwise continue_searching_right_matching_left[k_left]==0
    
    
    #3.1) k_right_matching_left
    
    #increment k_right_matching_left[k_left] if continue_searching_right_matching_left[k_left]==1
    circuit.append(incr_position.control(2),[continue_scanning,continue_searching_right_matching_left]+[k_right_matching_left[i] for i in range(nb_bits_position)])
    
    
    #3.2) continue_scanning
    
    #go to the next position
    #increment k_count
    circuit.append(incr_position,[k_count[i] for i in range(nb_bits_position)])
    
    #if the end has been reached, that is if k_count == 0
    #then do not continue_scanning
    circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[k_count[i] for i in range(nb_bits_position)]+[continue_scanning])
    
    
    #3.3) Transitions of loop_type
    
    #transitions grouped by end point
    
    #5 (.-(*)*(*)?   --[).]-> 7 (.-(*)*(*).-?
    #7 (.-(*)*(*).-? --[.)]-> 7 (.-(*)*(*).-) multi
    #7 (.-(*)*(*).-? <-[..]--
    
    #1 (.-(*)?       --[))]-> 6 (+(*))        bulge      INDIST-
    #5 (.-(*)*(*)?   --[))]-> 6 (+(*))        bulge      -INGUISHABLE -> increment 3rd bit
    
    #3 (.-(*).-?     --[.(]-> 5 (.-(*)*(*)?             INDIST-
    #7 (.-(*)*(*).-? --[.(]-> 5 (.-(*)*(*)?             -INGUISHABLE -> increment 3rd bit
    #1 (.-(*)?       --[)(]-> 5 (.-(*)*(*)?             INDIST-
    #5 (.-(*)*(*)?   <-[)(]--                           -INGUISHABLE -> increment 3rd bit
    
    #0 ((*)?         --[).]-> 4 ((*)+?                   INDIST-
    #4 ((*)+?        <-[*.]--                            -INGUISHABLE -> increment 3rd bit
    #0 ((*)?         --[)(]-> 4 ((*)+?                   INDIST-
    #4 ((*)+?        <-[*(]--                            -INGUISHABLE -> increment 3rd bit
    #4 ((*)+?        --[*)]-> 4 ((*)+)        bulge
    
    #1 (.-(*)?       --[).]-> 3 (.-(*).-?
    #3 (.-(*).-?     --[.)]-> 3 (.-(*).-)     interior
    #3 (.-(*).-?     <-[..]--
    
    #2 (.-?          --[.(]-> 1 (.-(*)?
    
    #0 (? init phase --[(.]-> 2 (.-?
    #0 (? init phase --[()]-> 2 (-)           hairpin
    #2 (.-?          --[.)]-> 2 (.-)          hairpin
    #2 (.-?          <-[..]--
    
    #0 (? init phase --[((]-> 0 ((*)?
    #0 ((*)?         --[))]-> 0 ((*))         stack
    
    
    #if k_folding == k_count
    start_if_equal(circuit,k_count,nb_bits_position,k_folding)
    
    if 0 == k:
        #3.3.1) init phase - the only possible state is 0
        #3.3.1.1) Transitions to loop_type=2
        #since 0 == k
        #then 0 == loop_type
        #so there is no need to control by loop_type
        
        #0 (? init phase --[(.]-> 2 (.-?
        #0 (? init phase --[()]-> 2 (-)           hairpin
        then_append(circuit,k_count,nb_bits_position,MCXGate(2+nb_bits_position+(nb_bits_bracket_dot-1),ctrl_state="0"+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[folding[0],loop_type[nb_bits_loop_type-1]])
        
        #3.3.1.2) Transitions to loop_type=0
        #0 (? init phase --[((]-> 0 ((*)?
        #do nothing
        
    #TODO in order to optimise
    #elif 1 == k:
    #    #3.3.2) possible states are 0,2
    #    #TODO
    #    pass
    #elif 2 == k:
    #    #3.3.3) possible states are 0,2,4,1
    #    #TODO
    #    pass
    #elif 3 == k:
    #    #3.3.4) possible states are 0,2,4,1,3,5,6
    #    #TODO
    #    pass
    else:
        #3.3.5) all states are possible
        #3.3.5.1) Transitions to loop_type=7 and loop_type=3
        #3.3.5.1.1) Detecting who should_transitionate to 7 or 3

        #1 (.-(*)?       --[).]-> 3 (.-(*).-?
        #5 (.-(*)*(*)?   --[).]-> 7 (.-(*)*(*).-?
        then_append(circuit,k_count,nb_bits_position,MCXGate(2+nb_bits_position+nb_bits_loop_type+nb_bits_bracket_dot,ctrl_state="00"+n_to_ctrl_state(1,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[folding[i] for i in range(nb_bits_bracket_dot)]+[should_transitionate])

        #3 (.-(*).-?     --[.)]-> 3 (.-(*).-)     interior
        #3 (.-(*).-?     <-[..]--
        #7 (.-(*)*(*).-? --[.)]-> 7 (.-(*)*(*).-) multi
        #7 (.-(*)*(*).-? <-[..]--
        #folding[k_count] == ) or . is characterised by 0 == folding[k_count][0]
        then_append(circuit,k_count,nb_bits_position,MCXGate(2+nb_bits_position+nb_bits_loop_type+1,ctrl_state="0"+n_to_ctrl_state(3,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[folding[0]]+[should_transitionate])


        #3.3.5.1.2) Transitionate by setting who should_transitionate to loop_type=7 ou loop_type=3

        #end if k_folding == k_count
        end_if_equal(circuit,k_count,nb_bits_position,k_folding)

        #decrement k_count
        circuit.append(decr_position,[k_count[i] for i in range(nb_bits_position)])

        #if k_folding == k_count
        start_if_equal(circuit,k_count,nb_bits_position,k_folding)

        #1 (.-(*)?       --[).]-> 3 (.-(*).-?
        #5 (.-(*)*(*)?   --[).]-> 7 (.-(*)*(*).-?
        start="".join(reversed(n_to_ctrl_state(1,nb_bits_loop_type))) #ctrl_state are in the wrong way
        end  ="".join(reversed(n_to_ctrl_state(3,nb_bits_loop_type))) #ctrl_state are in the wrong way
        for i in range(nb_bits_loop_type): #third_bit_loop_type is treated separately (and here there is nothing to do)
            if start[i] != end[i]:
                #this transition is characterised by [)*], that is by 1 == folding[k_count][nb_bits_bracket_dot-1]
                then_append(circuit,k_count,nb_bits_position,CCXGate(),[should_transitionate,folding[nb_bits_bracket_dot-1],loop_type[i]])

        #3 (.-(*).-?     --[.)]-> 3 (.-(*).-)     interior
        #3 (.-(*).-?     <-[..]--
        #7 (.-(*)*(*).-? --[.)]-> 7 (.-(*)*(*).-) multi
        #7 (.-(*)*(*).-? <-[..]--
        #do nothing

        #clean should_transitionate
        circuit.append(MCXGate(2+nb_bits_position+nb_bits_loop_type,ctrl_state=n_to_ctrl_state(3,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[should_transitionate])

        #end if k_folding == k_count
        end_if_equal(circuit,k_count,nb_bits_position,k_folding)

        #increment k_count
        circuit.append(incr_position,[k_count[i] for i in range(nb_bits_position)])





        #3.3.5.2) Transitions to loop_type=5
        #3.3.5.2.1) Detecting who should_transitionate to 5

        #if k_folding == k_count
        start_if_equal(circuit,k_count,nb_bits_position,k_folding)

        #3 (.-(*).-?     --[.(]-> 5 (.-(*)*(*)?             INDIST-
        #7 (.-(*)*(*).-? --[.(]-> 5 (.-(*)*(*)?             -INGUISHABLE -> increment 3rd bit
        #1 (.-(*)?       --[)(]-> 5 (.-(*)*(*)?             INDIST-
        #5 (.-(*)*(*)?   <-[)(]--                           -INGUISHABLE -> increment 3rd bit
        #and
        #7 (.-(*)*(*).-? --[.(]-> 5 (.-(*)*(*)?
        #should be differenciated from
        #5 (.-(*)*(*)?   --[).]-> 7 (.-(*)*(*).-?
        #7 (.-(*)*(*).-? --[.)]-> 7 (.-(*)*(*).-) multi
        #7 (.-(*)*(*).-? <-[..]--

        #folding[k_count] == ( is characterised by 1 == folding[k_count][0]
        #3 or 7 or 1 or 5 is characterised by 1 == loop_type[0]
        then_append(circuit,k_count,nb_bits_position,MCXGate(2+nb_bits_position+(nb_bits_loop_type-1)+(nb_bits_bracket_dot-1),ctrl_state="1"+n_to_ctrl_state(1,nb_bits_loop_type-1)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type-1)]+[folding[0],should_transitionate])


        #3.3.5.2.2) Transitionate by setting who should_transitionate to loop_type=5

        #end if k_folding == k_count
        end_if_equal(circuit,k_count,nb_bits_position,k_folding)

        #decrement k_count
        circuit.append(decr_position,[k_count[i] for i in range(nb_bits_position)])

        #if k_folding == k_count
        start_if_equal(circuit,k_count,nb_bits_position,k_folding)

        #3 (.-(*).-?     --[.(]-> 5 (.-(*)*(*)?             INDIST-
        #7 (.-(*)*(*).-? --[.(]-> 5 (.-(*)*(*)?             -INGUISHABLE -> increment 3rd bit
        start="".join(reversed(n_to_ctrl_state(3,nb_bits_loop_type)))
        end  ="".join(reversed(n_to_ctrl_state(5,nb_bits_loop_type)))
        for i in range(nb_bits_loop_type): #third_bit_loop_type is treated separately (and here there is nothing to do)
            if start[i] != end[i]:
                #this transition is characterised by [.*] (against [)*]) so by 0 == folding[k_count][nb_bits_bracket_dot-1]
                then_append(circuit,k_count,nb_bits_position,CCXGate(ctrl_state="01"),[should_transitionate,folding[nb_bits_bracket_dot-1],loop_type[i]])
        #(and increment third_bit_loop_type at the end)

        #1 (.-(*)?       --[)(]-> 5 (.-(*)*(*)?             INDIST-
        #5 (.-(*)*(*)?   <-[)(]--                           -INGUISHABLE -> increment 3rd bit
        #do nothing (except increment third_bit_loop_type at the end)

        #increment third_bit_loop_type
        circuit.append(incr_position.control(),[should_transitionate]+[third_bit_loop_type[i] for i in range(nb_bits_third_bit_loop_type)])

        #clean should_transitionate
        circuit.append(MCXGate(2+nb_bits_position+nb_bits_loop_type,ctrl_state=n_to_ctrl_state(5,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[should_transitionate])

        #end if k_folding == k_count
        end_if_equal(circuit,k_count,nb_bits_position,k_folding)

        #increment k_count
        circuit.append(incr_position,[k_count[i] for i in range(nb_bits_position)])





        #3.3.5.3) Transitions to loop_type=6 and loop_type=1
        #3.3.5.3.1) Detecting who should_transitionate to 6

        #if k_folding == k_count
        start_if_equal(circuit,k_count,nb_bits_position,k_folding)

        #2 (.-?          --[.(]-> 1 (.-(*)?
        #there is no ned to differenciate 2 from 6 because once 6 is reached, 0 == continue_searching_right_matching_left
        then_append(circuit,k_count,nb_bits_position,MCXGate(2+nb_bits_position+nb_bits_loop_type+(nb_bits_bracket_dot-1),ctrl_state="1"+n_to_ctrl_state(2,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[folding[nb_bits_bracket_dot-1],should_transitionate])

        #1 (.-(*)?       --[))]-> 6 (+(*))        bulge      INDIST-
        #5 (.-(*)*(*)?   --[))]-> 6 (+(*))        bulge      -INGUISHABLE -> increment 3rd bit
        #folding[k_count] == ) is characterised by 1 == folding[k_count][1]
        #1 or 5 is characterised by 0 == loop_type[1] and 1 == loop_type[0]
        then_append(circuit,k_count,nb_bits_position,MCXGate(2+nb_bits_position+nb_bits_loop_type+(nb_bits_bracket_dot-1),ctrl_state="1"+n_to_ctrl_state(1,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[folding[0],should_transitionate])


        #3.3.5.3.2) Transitionate by setting who should_transitionate to loop_type=6

        #should do the same thing whatever the transition (to 6 or to 1), so there is no need to control by folding[k_folding-1]

        #2 (.-?          --[.(]-> 1 (.-(*)?
        #1 (.-(*)?       --[))]-> 6 (+(*))        bulge      INDIST-
        #5 (.-(*)*(*)?   --[))]-> 6 (+(*))        bulge      -INGUISHABLE -> increment 3rd bit
        start="".join(reversed(n_to_ctrl_state(5,nb_bits_loop_type))) #ctrl_state are in the wrong way
        end  ="".join(reversed(n_to_ctrl_state(6,nb_bits_loop_type))) #ctrl_state are in the wrong way
        for i in range(nb_bits_loop_type): #third_bit_loop_type is treated separately
            if start[i] != end[i]:
                circuit.cx(should_transitionate,loop_type[i])

        #increment third_bit_loop_type if transitionate to 6
        #that is if should transitionate and ) == folding[k_count]
        circuit.append(incr_position.control(2),[should_transitionate,folding[nb_bits_bracket_dot-1]]+[third_bit_loop_type[i] for i in range(nb_bits_third_bit_loop_type)])

        #note that the transition 1 --[*(]-> has aleady been treated, to transitionning to 1 won't pause any problem

        #clean should_transitionate
        circuit.append(MCXGate(2+nb_bits_position+nb_bits_loop_type,ctrl_state=n_to_ctrl_state(6,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[should_transitionate])





        #3.3.5.4) Transitions to loop_type=4

        #0 ((*)?         --[).]-> 4 ((*)+?                   INDIST-
        #4 ((*)+?        <-[*.]--                            -INGUISHABLE -> increment 3rd bit
        #0 ((*)?         --[)(]-> 4 ((*)+?                   INDIST-
        #4 ((*)+?        <-[*(]--                            -INGUISHABLE -> increment 3rd bit
        #4 ((*)+?        --[*)]-> 4 ((*)+)        bulge
        #use an ancillary qbit could_transitionate to detect the case 0 --[*)]-> 4 (the only case that  is not in the list above)
        circuit.x(could_transitionate)
        then_append(circuit,k_count,nb_bits_position,MCXGate((nb_bits_loop_type+nb_bits_third_bit_loop_type)+1,ctrl_state="1"+n_to_ctrl_state(0,nb_bits_loop_type+nb_bits_third_bit_loop_type)),[loop_type[i] for i in range(nb_bits_loop_type)]+[third_bit_loop_type[i] for i in range(nb_bits_third_bit_loop_type)]+[folding[nb_bits_bracket_dot-1],could_transitionate])

        #everyone verifying 0 == loop_type[0] and 1 == loop_type[1] transitionates (that is increment third_bit_loop_type) as far as they could tansitionate
        circuit.append(incr_position.control(2+nb_bits_position+nb_bits_loop_type+1,ctrl_state="1"+n_to_ctrl_state(0,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[could_transitionate]+[third_bit_loop_type[i] for i in range(nb_bits_third_bit_loop_type)])

        #clean could_transitionate
        #no more need to control by ) == folding[k_count] since the only 0 == loop_type that stay verify it
        #TO VERIFY : it seems to me that there is a need of then_append() here, even though there is no control by folding[k_count], but I am not sure
        then_append(circuit,k_count,nb_bits_position,MCXGate(nb_bits_loop_type+nb_bits_third_bit_loop_type,ctrl_state=n_to_ctrl_state(0,nb_bits_loop_type+nb_bits_third_bit_loop_type)),[loop_type[i] for i in range(nb_bits_loop_type)]+[third_bit_loop_type[i] for i in range(nb_bits_third_bit_loop_type)]+[could_transitionate])
        circuit.x(could_transitionate)





        #3.3.5.5) Transitions to loop_type=2 and loop_type=0
        #2 (.-?          --[.)]-> 2 (.-)          hairpin
        #2 (.-?          <-[..]--
        #0 ((*)?         --[))]-> 0 ((*))         stack
        #do nothing

    #3.4) Update count[k_left]

    #if continue_searching_right_matching_left[k_left]==1
    #then update count[k_left] for the next candidate
    if 0 == k:
        #necessarily if folding[k_left]==( then continue_searching_right_matching_left[k_left]==1
        #so it is ok not controlling by continue_searching_right_matching_left here
        #( is characterised by 1 == folding[k_count][0]
        then_append(circuit,k_count,nb_bits_position,incr_position.control(2,ctrl_state="11"),[continue_scanning,folding[0                    ]]+[count[i] for i in range(nb_bits_position)])
        #) is characterised by 1 == folding[nb_bits_bracket_dot-1]
        then_append(circuit,k_count,nb_bits_position,decr_position.control(2,ctrl_state="11"),[continue_scanning,folding[nb_bits_bracket_dot-1]]+[count[i] for i in range(nb_bits_position)])
    else:
        #( is characterised by 1 == folding[k_count][0]
        then_append(circuit,k_count,nb_bits_position,incr_position.control(3,ctrl_state="111"),[continue_scanning,continue_searching_right_matching_left,folding[0                    ]]+[count[i] for i in range(nb_bits_position)])
        #) is characterised by 1 == folding[nb_bits_bracket_dot-1]
        #so there is no need controling by folding[0]
        then_append(circuit,k_count,nb_bits_position,decr_position.control(3,ctrl_state="111"),[continue_scanning,continue_searching_right_matching_left,folding[nb_bits_bracket_dot-1]]+[count[i] for i in range(nb_bits_position)])

    #if continue_searching_right_matching_left[k_left]==0, then simply increment count[k_left] so that it never equals 0 again
    #it is ok not controlling by continue_scanning here
    if 0 != k:
        #as stated previously, if 0==k then necessarily if folding[k_left]==( then continue_searching_right_matching_left[k_left]==1
        #so there is no need doing this if 0==k
        then_append(circuit,k_count,nb_bits_position,incr_position.control(ctrl_state="0"),[continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)])

    #if 1 == count[k_left]
    start_if_equal_const(circuit,count,nb_bits_position,1)


    #3.5) Update loop[k_left]

    #if 1 == continue_searching_right_matching_left[k_left] then increment loop[k_left]
    if 0==k:
        #since the value 0==loop[k_left] is known then there is no need to incr_position in order to increment
        #a CXGate() is sufficient
        then_append(circuit,[k_count[i] for i in range(nb_bits_position)]+[count[i] for i in range(nb_bits_position)],2*nb_bits_position,CCXGate(),[continue_scanning,continue_searching_right_matching_left,loop[0]])
    else:
        then_append(circuit,[k_count[i] for i in range(nb_bits_position)]+[count[i] for i in range(nb_bits_position)],2*nb_bits_position,incr_position.control(2),[continue_scanning,continue_searching_right_matching_left]+[loop[i] for i in range(nb_bits_position)])

    #end if 1 == count[k_left]
    end_if_equal_const(circuit,count,nb_bits_position,1)

    #end if k_folding == k_count
    end_if_equal(circuit,k_count,nb_bits_position,k_folding)


    #3.6) Update continue_searching_right_matching_left

    #if count[k_left] == 0
    #then there is no matching ) to find
    circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[count[i] for i in range(nb_bits_position)]+[continue_searching_right_matching_left])
    
    return circuit

def step_gate(nb_bits_position,incr_position,decr_position,nb_bits_third_bit_loop_type):
    return step_circuit(nb_bits_position,incr_position,decr_position,nb_bits_third_bit_loop_type).to_gate(label="step")

    
def matching_parenthesis_circuit(length,truncation_point=50):
    nb_bits_position=ceil(log2(length+1)) #length+1 because we add a ( at the begining to check the folding is well formed
    #add_positions=add_gate(nb_bits_position)
    incr_position=incr_gate(nb_bits_position)
    decr_position=decr_gate(nb_bits_position)
    #nb_bits_extend = max(0,nb_bits_position-nb_bits_bracket_dot)
    nb_bits_third_bit_loop_type = nb_bits_position
    
    step = step_gate(nb_bits_position,incr_position,decr_position,nb_bits_third_bit_loop_type)
    
    #input
    #semantics of folding[0:nb_bits_bracket_dot]
    #. == "00"
    #h == "11"
    #) == "10"
    #( == "01"
    folding = QuantumRegister(nb_bits_bracket_dot,name="(/./h/)")
    
    #auxillary qbits
    k_folding = QuantumRegister(nb_bits_position,name="k_folding")
    count = QuantumRegister(nb_bits_position,name="count") #count how many parenthesis still need to be closed. folding[k_folding=0] is initiallised to ( and count[k_count=0] is used to check that there is no extra ) that would match in the folding
    k_count = QuantumRegister(nb_bits_position,name="k_count")
    continue_searching_right_matching_left = QuantumRegister(1,name="continue_match") #continue_searching_right_matching_left[k] corresponds to count[k]
    
    change_folding_encoding = QuantumRegister(1,name="change_encoding")
    continue_scanning = QuantumRegister(1,name="continue_scan")
    zero = QuantumRegister(1,name="zero")
    #one = QuantumRegister(1,name="one")
    
    #results
    k_right_matching_left = QuantumRegister(nb_bits_position,name="k_match") #k_right_matching_left[k_left] is initialised to k_left and is incremented until count[k_left]==0, that is until k_right_matching_left[k_left] is the indice of the ) matching the ( == folding[k_left]
    loop = QuantumRegister(nb_bits_position,name="loop") #length of the loop starting at folding[k_left]==(. loop[k_left] is initialised to 0 and is incremented each time count[k_left]==1 until count[k_left]==0
    
    #semantics of loop_type
    #where * is anything well matched of any lenght (including 0)
    #      + is anything well matched of length > 0
    #      - is any number of dots (including 0)
    #0 init phase (?       / scan phase ((*)?             / stack    ((*))
    #1                       scan phase (.-(*)?
    #2                       scan phase (.-?              / hairpin  (-)
    #3                       scan phase (.-(*).-?         / interior (.-(*).-)
    #4                       scan phase ((*)+?            / bulge    ((*)+)
    #5                       scan phase (.-(*)*(*)?
    #6                                                      bulge    (+(*))
    #7                       scan phase (.-(*)*(*).-?     / multi    (.-(*)*(*).-)
    
    #so the loop_type is coded on 3 bits,
    #but actually, the last bit is coded by loop_type[nb_bits_loop_type:nb_bits_loop_type+nb_bits_position] with the folowing semantics
    #loop_type[nb_bits_loop_type:nb_bits_loop_type+nb_bits_position] -> 3rd bit of loop_type
    #0                                                               -> 0
    #!= 0                                                            -> 1
    
    #transitions
    # -> transition to another state
    # <- transition to oneself
    #transition only if continue_searching_right_matching_left
    #when at a level >=2 just wait until beeing again at level 1
    
    
    #transitions grouped by start point
    
    #init phase only
    #0 (?            --[((]-> 0 ((*)?
    #0 (?            --[(.]-> 2 (.-?
    #0 (?            --[()]-> 2 (-)           hairpin
    
    #scan phase
    #0 ((*)?         --[))]-> 0 ((*))         stack
    #0 ((*)?         --[).]-> 4 ((*)+?
    #0 ((*)?         --[)(]-> 4 ((*)+?
    
    #1 (.-(*)?       --[).]-> 3 (.-(*).-?
    #1 (.-(*)?       --[))]-> 6 (+(*))        bulge
    #1 (.-(*)?       --[)(]-> 5 (.-(*)*(*)?
    
    #2 (.-?          <-[..]--
    #2 (.-?          --[.(]-> 1 (.-(*)?
    #2 (.-?          --[.)]-> 2 (-)           hairpin
    
    #3 (.-(*).-?     <-[..]--
    #3 (.-(*).-?     --[.)]-> 3 (.-(*).-)     interior
    #3 (.-(*).-?     --[.(]-> 5 (.-(*)*(*)?
    
    #once 3rd bit have been set to 1, it will neve be set to 0 again
    #incerment 3rd bit every time a transition to n>=4 occurs if tit is possible to transitionate from m<=3 to n
    
    #4 ((*)+?        --[*)]-> 4 ((*)+)        bulge
    #4 ((*)+?        <-[*.]--
    #4 ((*)+?        <-[*(]--
    
    #5 (.-(*)*(*)?   <-[)(]--
    #5 (.-(*)*(*)?   --[))]-> 6 (+(*))        bulge
    #5 (.-(*)*(*)?   --[).]-> 7 (.-(*)*(*).-?
    
    #7 (.-(*)*(*).-? <-[..]--
    #7 (.-(*)*(*).-? --[.(]-> 5 (.-(*)*(*)?
    #7 (.-(*)*(*).-? --[.)]-> 7 (.-(*)*(*).-) multi
    
    
    #transitions grouped by end point
    #0 (? init phase --[((]-> 0 ((*)?
    #0 ((*)?         --[))]-> 0 ((*))         stack
    
    #2 (.-?          --[.(]-> 1 (.-(*)?
    
    #0 (? init phase --[(.]-> 2 (.-?
    #0 (? init phase --[()]-> 2 (-)           hairpin
    #2 (.-?          --[.)]-> 2 (.-)          hairpin
    #2 (.-?          <-[..]--
    
    #1 (.-(*)?       --[).]-> 3 (.-(*).-?
    #3 (.-(*).-?     --[.)]-> 3 (.-(*).-)     interior
    #3 (.-(*).-?     <-[..]--
    
    #0 ((*)?         --[).]-> 4 ((*)+?                   INDIST-
    #4 ((*)+?        <-[*.]--                            -INGUISHABLE -> increment 3rd bit
    #0 ((*)?         --[)(]-> 4 ((*)+?                   INDIST-
    #4 ((*)+?        <-[*(]--                            -INGUISHABLE -> increment 3rd bit
    #4 ((*)+?        --[*)]-> 4 ((*)+)        bulge
    
    #3 (.-(*).-?     --[.(]-> 5 (.-(*)*(*)?             INDIST-
    #7 (.-(*)*(*).-? --[.(]-> 5 (.-(*)*(*)?             -INGUISHABLE -> increment 3rd bit
    #1 (.-(*)?       --[)(]-> 5 (.-(*)*(*)?             INDIST-
    #5 (.-(*)*(*)?   <-[)(]--                           -INGUISHABLE -> increment 3rd bit
    
    #1 (.-(*)?       --[))]-> 6 (+(*))        bulge      INDIST-
    #5 (.-(*)*(*)?   --[))]-> 6 (+(*))        bulge      -INGUISHABLE -> increment 3rd bit
    
    #5 (.-(*)*(*)?   --[).]-> 7 (.-(*)*(*).-?
    #7 (.-(*)*(*).-? --[.)]-> 7 (.-(*)*(*).-) multi
    #7 (.-(*)*(*).-? <-[..]--
    
    loop_type = QuantumRegister(nb_bits_loop_type,name="loop_type")
    third_bit_loop_type = QuantumRegister(nb_bits_third_bit_loop_type,name="loop_type[{}]_encoding)".format(nb_bits_loop_type))
    incorrect_folding = QuantumRegister(nb_bits_position,name="incorrect")
    correct_folding = QuantumRegister(1,name="correct")
    
    #circuit = QuantumCircuit(folding,k_folding,change_folding_encoding,k_count,count,k_right_matching_left,zero,continue_searching_right_matching_left,continue_scanning,loop,incorrect_folding,correct_folding) #order of registers as they appear along the program
    circuit = QuantumCircuit()
    
    def semantics(bit_string):
        i_bit = 0
        semantics_string = ""
        
        def semantics_number(label,nb_bits_number):
            s=str(int(bit_string[i_bit:i_bit+nb_bits_number],2))
            delta=nb_bits_number-len(s)
            return s+"_"+label+delta*" "+" ",nb_bits_number
        
        if circuit.has_register(correct_folding):
            s,i=semantics_number("?",1)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(incorrect_folding):
            s,i=semantics_number("!",nb_bits_position)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(loop_type):
            s,i=semantics_number("lt",nb_bits_loop_type)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(loop):
            s,i=semantics_number("lo",nb_bits_position)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(continue_scanning):
            s,i=semantics_number("sc",1)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(continue_searching_right_matching_left):
            s,i=semantics_number("se",1)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(zero):
            semantics_string+="cst"
            semantics_string+=bit_string[i_bit:i_bit+1]
            i_bit+=1
        #if circuit.has_register(one):
        #    semantics_string+=bit_string[i_bit:i_bit+1]
        #    i_bit+=1
            semantics_string+=" "
        if circuit.has_register(k_right_matching_left):
            s,i=semantics_number("km",nb_bits_position)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(count):
            s,i=semantics_number("c",nb_bits_position)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(k_count):
            s,i=semantics_number("kc",nb_bits_position)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(change_folding_encoding):
            semantics_string+="-" if bit_string[i_bit:i_bit+1]=="0" else "h"
            i_bit+=1
            semantics_string+=" "
        if circuit.has_register(k_folding):
            s,i=semantics_number("kf",nb_bits_position)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(folding):
            semantics_string+=("." if bit_string[i_bit:i_bit+nb_bits_bracket_dot]=="00" else
                               "h" if bit_string[i_bit:i_bit+nb_bits_bracket_dot]=="11" else
                               "(" if bit_string[i_bit:i_bit+nb_bits_bracket_dot]=="01" else
                               ")" if bit_string[i_bit:i_bit+nb_bits_bracket_dot]=="10" else "")
            i_bit+=nb_bits_bracket_dot
        return semantics_string

    
    
    #1) Superposition at the begining of the Grover search
    
    #any symbol
    circuit.add_register(folding)
    for i in range(nb_bits_bracket_dot):
        circuit.h(folding[i])
    
    if  truncation_point == "any folding" or truncation_point <= 0:
        return circuit,semantics
    
    #at any position
    circuit.add_register(k_folding)
    circuit.h(k_folding)
    
    if truncation_point == "any k_folding" or truncation_point <= 1:
        return circuit,semantics
    
    #except at position n_to_ctrl_state(0,nb_bits_position), where it is necessarily a ( == "01" to check if the folding is well formed
    for i in range(nb_bits_bracket_dot):
        circuit.append(HGate().control(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[k_folding[j] for j in range(nb_bits_position)]+[folding[i]])
    circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[k_folding[j] for j in range(nb_bits_position)]+[folding[0]])
    
    #except at positions k > length, where folding[k] is set to .
    for k in range(length+1,2**nb_bits_position):
        for i in range(nb_bits_bracket_dot):
            circuit.append(HGate().control(nb_bits_position,ctrl_state=n_to_ctrl_state(k,nb_bits_position)),[k_folding[j] for j in range(nb_bits_position)]+[folding[i]])
    
    if truncation_point == "candidate folding" or truncation_point <= 2:
        return circuit,semantics
    
    circuit.add_register(change_folding_encoding)
    
    #if this symbol is h == "11", then change it to be . == "00"
    circuit.ccx(*[folding[i] for i in range(nb_bits_bracket_dot)],change_folding_encoding)
    circuit.append(MCMT(XGate(),1,2),[change_folding_encoding]+[folding[i] for i in range(nb_bits_bracket_dot)])
    
    if truncation_point == "encoding" or truncation_point <= 3:
        return circuit,semantics
    
    
    #2) Initialisation of the oracle
    
    #initialise k_count to k_left
    circuit.add_register(k_count)
    circuit.h(k_count)
    
    if truncation_point == "init k_count" or truncation_point <= 4:
        return circuit,semantics
    
    #if k_folding == k_count
    start_if_equal(circuit,k_count,nb_bits_position,k_folding)
    
    #initialise count with folding
    circuit.add_register(count)
    #folding -> count
    #. == "00" -> 0 == "0...000"
    #do nothing
    #) == "10" -> 0 == "0...000"
    #do nothing
    #( == "01" -> 1 == "0...001"
    then_append(circuit,k_count,nb_bits_position,CXGate(),[folding[0],count[0]])
    
    #end if k_folding == k_count
    end_if_equal(circuit,k_count,nb_bits_position,k_folding)
    
    if truncation_point == "init count" or truncation_point <= 5:
        return circuit,semantics
    
    #initialise k_right_matching_left[k_count] to k_left (and it will then be incremented up to the matchning k_right)
    circuit.add_register(k_right_matching_left)
    for i in range(nb_bits_position):
        circuit.cnot(k_count[i],k_right_matching_left[i])
    
    if truncation_point == "init k_right_matching_left" or truncation_point <= 6:
        return circuit,semantics
    
    #initialise one to 1
    #circuit.add_register(one)
    circuit.add_register(zero)
    #circuit.x(one)
    
    if truncation_point == "init cst" or truncation_point <= 7:
        return circuit,semantics
    
    #initialise continue_searching_right_matching_left=1 when 1 == count
    #indeed, when folding[k_left] == ( (which will be checked by there <*>) then there is initially 1 parenthesis to be closed
    circuit.add_register(continue_searching_right_matching_left)
    circuit.cnot(count[0],continue_searching_right_matching_left)
    
    if truncation_point == "init continue_searching_right_matching_left" or truncation_point <= 8:
        return circuit,semantics
    
    #initialise continue_scanning=1
    circuit.add_register(continue_scanning)
    circuit.x(continue_scanning)
    
    if truncation_point == "init continue_scanning" or truncation_point <= 9:
        return circuit,semantics
    
    circuit.add_register(loop)
    circuit.add_register(loop_type)
    circuit.add_register(third_bit_loop_type)
    #ancillary qbit for computing loop_type
    #it is cleaned every time after use
    #zero serves as should_transitionate
    should_transitionate=zero
    #should_transitionate serves as should_transitionate
    could_transitionate=should_transitionate
    
    #when needing to control by 0 != third_bit_loop_type
    #get an ancillary qbit loop_type[nb_bits_loop_type] and encode it by third_bit_loop_type like this
    #circuit.append(MCXGate(nb_bits_position,n_to_ctrl_state(0,nb_bits_position)),[third_bit_loop_type[i] for i in range(nb_bits_third_bit_loop_type)]+[loop_type[nb_bits_loop_type]])
    #but actually it never happens
    #in case it happens one day, remember to clean it before modifying third_bit_loop_type
    
    
    #3) Compute parameters by scanning the folding
    
    #for k in range(length):
    #instead, do the optimised k=0 case and then just use the step gate
    k=0

    if truncation_point <= 10 + k:
        return circuit,semantics

    #<*> if it has been checked that there is a matching ) to find, then continue_searching_right_matching_left[k_left]==1, otherwise continue_searching_right_matching_left[k_left]==0


    #3.1) k_right_matching_left

    #increment k_right_matching_left[k_left] if continue_searching_right_matching_left[k_left]==1
    circuit.append(incr_position.control(2),[continue_scanning,continue_searching_right_matching_left]+[k_right_matching_left[i] for i in range(nb_bits_position)])


    #3.2) continue_scanning

    #go to the next position
    #increment k_count
    circuit.append(incr_position,[k_count[i] for i in range(nb_bits_position)])

    #if the end has been reached, that is if k_count == 0
    #then do not continue_scanning
    circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[k_count[i] for i in range(nb_bits_position)]+[continue_scanning])


    #3.3) Transitions of loop_type

    #transitions grouped by end point

    #5 (.-(*)*(*)?   --[).]-> 7 (.-(*)*(*).-?
    #7 (.-(*)*(*).-? --[.)]-> 7 (.-(*)*(*).-) multi
    #7 (.-(*)*(*).-? <-[..]--

    #1 (.-(*)?       --[))]-> 6 (+(*))        bulge      INDIST-
    #5 (.-(*)*(*)?   --[))]-> 6 (+(*))        bulge      -INGUISHABLE -> increment 3rd bit

    #3 (.-(*).-?     --[.(]-> 5 (.-(*)*(*)?             INDIST-
    #7 (.-(*)*(*).-? --[.(]-> 5 (.-(*)*(*)?             -INGUISHABLE -> increment 3rd bit
    #1 (.-(*)?       --[)(]-> 5 (.-(*)*(*)?             INDIST-
    #5 (.-(*)*(*)?   <-[)(]--                           -INGUISHABLE -> increment 3rd bit

    #0 ((*)?         --[).]-> 4 ((*)+?                   INDIST-
    #4 ((*)+?        <-[*.]--                            -INGUISHABLE -> increment 3rd bit
    #0 ((*)?         --[)(]-> 4 ((*)+?                   INDIST-
    #4 ((*)+?        <-[*(]--                            -INGUISHABLE -> increment 3rd bit
    #4 ((*)+?        --[*)]-> 4 ((*)+)        bulge

    #1 (.-(*)?       --[).]-> 3 (.-(*).-?
    #3 (.-(*).-?     --[.)]-> 3 (.-(*).-)     interior
    #3 (.-(*).-?     <-[..]--

    #2 (.-?          --[.(]-> 1 (.-(*)?

    #0 (? init phase --[(.]-> 2 (.-?
    #0 (? init phase --[()]-> 2 (-)           hairpin
    #2 (.-?          --[.)]-> 2 (.-)          hairpin
    #2 (.-?          <-[..]--

    #0 (? init phase --[((]-> 0 ((*)?
    #0 ((*)?         --[))]-> 0 ((*))         stack


    #if k_folding == k_count
    start_if_equal(circuit,k_count,nb_bits_position,k_folding)

    if 0 == k:
        #3.3.1) init phase - the only possible state is 0
        #3.3.1.1) Transitions to loop_type=2
        #since 0 == k
        #then 0 == loop_type
        #so there is no need to control by loop_type

        #0 (? init phase --[(.]-> 2 (.-?
        #0 (? init phase --[()]-> 2 (-)           hairpin
        then_append(circuit,k_count,nb_bits_position,MCXGate(2+nb_bits_position+(nb_bits_bracket_dot-1),ctrl_state="0"+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[folding[0],loop_type[nb_bits_loop_type-1]])

        #3.3.1.2) Transitions to loop_type=0
        #0 (? init phase --[((]-> 0 ((*)?
        #do nothing

    #TODO in order to optimise
    #elif 1 == k:
    #    #3.3.2) possible states are 0,2
    #    #TODO
    #    pass
    #elif 2 == k:
    #    #3.3.3) possible states are 0,2,4,1
    #    #TODO
    #    pass
    #elif 3 == k:
    #    #3.3.4) possible states are 0,2,4,1,3,5,6
    #    #TODO
    #    pass
    else:
        #3.3.5) all states are possible
        #3.3.5.1) Transitions to loop_type=7 and loop_type=3
        #3.3.5.1.1) Detecting who should_transitionate to 7 or 3

        #1 (.-(*)?       --[).]-> 3 (.-(*).-?
        #5 (.-(*)*(*)?   --[).]-> 7 (.-(*)*(*).-?
        then_append(circuit,k_count,nb_bits_position,MCXGate(2+nb_bits_position+nb_bits_loop_type+nb_bits_bracket_dot,ctrl_state="00"+n_to_ctrl_state(1,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[folding[i] for i in range(nb_bits_bracket_dot)]+[should_transitionate])

        #3 (.-(*).-?     --[.)]-> 3 (.-(*).-)     interior
        #3 (.-(*).-?     <-[..]--
        #7 (.-(*)*(*).-? --[.)]-> 7 (.-(*)*(*).-) multi
        #7 (.-(*)*(*).-? <-[..]--
        #folding[k_count] == ) or . is characterised by 0 == folding[k_count][0]
        then_append(circuit,k_count,nb_bits_position,MCXGate(2+nb_bits_position+nb_bits_loop_type+1,ctrl_state="0"+n_to_ctrl_state(3,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[folding[0]]+[should_transitionate])


        #3.3.5.1.2) Transitionate by setting who should_transitionate to loop_type=7 ou loop_type=3

        #end if k_folding == k_count
        end_if_equal(circuit,k_count,nb_bits_position,k_folding)

        #decrement k_count
        circuit.append(decr_position,[k_count[i] for i in range(nb_bits_position)])

        #if k_folding == k_count
        start_if_equal(circuit,k_count,nb_bits_position,k_folding)

        #1 (.-(*)?       --[).]-> 3 (.-(*).-?
        #5 (.-(*)*(*)?   --[).]-> 7 (.-(*)*(*).-?
        start="".join(reversed(n_to_ctrl_state(1,nb_bits_loop_type))) #ctrl_state are in the wrong way
        end  ="".join(reversed(n_to_ctrl_state(3,nb_bits_loop_type))) #ctrl_state are in the wrong way
        for i in range(nb_bits_loop_type): #third_bit_loop_type is treated separately (and here there is nothing to do)
            if start[i] != end[i]:
                #this transition is characterised by [)*], that is by 1 == folding[k_count][nb_bits_bracket_dot-1]
                then_append(circuit,k_count,nb_bits_position,CCXGate(),[should_transitionate,folding[nb_bits_bracket_dot-1],loop_type[i]])

        #3 (.-(*).-?     --[.)]-> 3 (.-(*).-)     interior
        #3 (.-(*).-?     <-[..]--
        #7 (.-(*)*(*).-? --[.)]-> 7 (.-(*)*(*).-) multi
        #7 (.-(*)*(*).-? <-[..]--
        #do nothing

        #clean should_transitionate
        circuit.append(MCXGate(2+nb_bits_position+nb_bits_loop_type,ctrl_state=n_to_ctrl_state(3,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[should_transitionate])

        #end if k_folding == k_count
        end_if_equal(circuit,k_count,nb_bits_position,k_folding)

        #increment k_count
        circuit.append(incr_position,[k_count[i] for i in range(nb_bits_position)])





        #3.3.5.2) Transitions to loop_type=5
        #3.3.5.2.1) Detecting who should_transitionate to 5

        #if k_folding == k_count
        start_if_equal(circuit,k_count,nb_bits_position,k_folding)

        #3 (.-(*).-?     --[.(]-> 5 (.-(*)*(*)?             INDIST-
        #7 (.-(*)*(*).-? --[.(]-> 5 (.-(*)*(*)?             -INGUISHABLE -> increment 3rd bit
        #1 (.-(*)?       --[)(]-> 5 (.-(*)*(*)?             INDIST-
        #5 (.-(*)*(*)?   <-[)(]--                           -INGUISHABLE -> increment 3rd bit
        #and
        #7 (.-(*)*(*).-? --[.(]-> 5 (.-(*)*(*)?
        #should be differenciated from
        #5 (.-(*)*(*)?   --[).]-> 7 (.-(*)*(*).-?
        #7 (.-(*)*(*).-? --[.)]-> 7 (.-(*)*(*).-) multi
        #7 (.-(*)*(*).-? <-[..]--

        #folding[k_count] == ( is characterised by 1 == folding[k_count][0]
        #3 or 7 or 1 or 5 is characterised by 1 == loop_type[0]
        then_append(circuit,k_count,nb_bits_position,MCXGate(2+nb_bits_position+(nb_bits_loop_type-1)+(nb_bits_bracket_dot-1),ctrl_state="1"+n_to_ctrl_state(1,nb_bits_loop_type-1)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type-1)]+[folding[0],should_transitionate])


        #3.3.5.2.2) Transitionate by setting who should_transitionate to loop_type=5

        #end if k_folding == k_count
        end_if_equal(circuit,k_count,nb_bits_position,k_folding)

        #decrement k_count
        circuit.append(decr_position,[k_count[i] for i in range(nb_bits_position)])

        #if k_folding == k_count
        start_if_equal(circuit,k_count,nb_bits_position,k_folding)

        #3 (.-(*).-?     --[.(]-> 5 (.-(*)*(*)?             INDIST-
        #7 (.-(*)*(*).-? --[.(]-> 5 (.-(*)*(*)?             -INGUISHABLE -> increment 3rd bit
        start="".join(reversed(n_to_ctrl_state(3,nb_bits_loop_type)))
        end  ="".join(reversed(n_to_ctrl_state(5,nb_bits_loop_type)))
        for i in range(nb_bits_loop_type): #third_bit_loop_type is treated separately (and here there is nothing to do)
            if start[i] != end[i]:
                #this transition is characterised by [.*] (against [)*]) so by 0 == folding[k_count][nb_bits_bracket_dot-1]
                then_append(circuit,k_count,nb_bits_position,CCXGate(ctrl_state="01"),[should_transitionate,folding[nb_bits_bracket_dot-1],loop_type[i]])
        #(and increment third_bit_loop_type at the end)

        #1 (.-(*)?       --[)(]-> 5 (.-(*)*(*)?             INDIST-
        #5 (.-(*)*(*)?   <-[)(]--                           -INGUISHABLE -> increment 3rd bit
        #do nothing (except increment third_bit_loop_type at the end)

        #increment third_bit_loop_type
        circuit.append(incr_position.control(),[should_transitionate]+[third_bit_loop_type[i] for i in range(nb_bits_third_bit_loop_type)])

        #clean should_transitionate
        circuit.append(MCXGate(2+nb_bits_position+nb_bits_loop_type,ctrl_state=n_to_ctrl_state(5,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[should_transitionate])

        #end if k_folding == k_count
        end_if_equal(circuit,k_count,nb_bits_position,k_folding)

        #increment k_count
        circuit.append(incr_position,[k_count[i] for i in range(nb_bits_position)])





        #3.3.5.3) Transitions to loop_type=6 and loop_type=1
        #3.3.5.3.1) Detecting who should_transitionate to 6

        #if k_folding == k_count
        start_if_equal(circuit,k_count,nb_bits_position,k_folding)

        #2 (.-?          --[.(]-> 1 (.-(*)?
        #there is no ned to differenciate 2 from 6 because once 6 is reached, 0 == continue_searching_right_matching_left
        then_append(circuit,k_count,nb_bits_position,MCXGate(2+nb_bits_position+nb_bits_loop_type+(nb_bits_bracket_dot-1),ctrl_state="1"+n_to_ctrl_state(2,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[folding[nb_bits_bracket_dot-1],should_transitionate])

        #1 (.-(*)?       --[))]-> 6 (+(*))        bulge      INDIST-
        #5 (.-(*)*(*)?   --[))]-> 6 (+(*))        bulge      -INGUISHABLE -> increment 3rd bit
        #folding[k_count] == ) is characterised by 1 == folding[k_count][1]
        #1 or 5 is characterised by 0 == loop_type[1] and 1 == loop_type[0]
        then_append(circuit,k_count,nb_bits_position,MCXGate(2+nb_bits_position+nb_bits_loop_type+(nb_bits_bracket_dot-1),ctrl_state="1"+n_to_ctrl_state(1,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[folding[0],should_transitionate])


        #3.3.5.3.2) Transitionate by setting who should_transitionate to loop_type=6

        #should do the same thing whatever the transition (to 6 or to 1), so there is no need to control by folding[k_folding-1]

        #2 (.-?          --[.(]-> 1 (.-(*)?
        #1 (.-(*)?       --[))]-> 6 (+(*))        bulge      INDIST-
        #5 (.-(*)*(*)?   --[))]-> 6 (+(*))        bulge      -INGUISHABLE -> increment 3rd bit
        start="".join(reversed(n_to_ctrl_state(5,nb_bits_loop_type))) #ctrl_state are in the wrong way
        end  ="".join(reversed(n_to_ctrl_state(6,nb_bits_loop_type))) #ctrl_state are in the wrong way
        for i in range(nb_bits_loop_type): #third_bit_loop_type is treated separately
            if start[i] != end[i]:
                circuit.cx(should_transitionate,loop_type[i])

        #increment third_bit_loop_type if transitionate to 6
        #that is if should transitionate and ) == folding[k_count]
        circuit.append(incr_position.control(2),[should_transitionate,folding[nb_bits_bracket_dot-1]]+[third_bit_loop_type[i] for i in range(nb_bits_third_bit_loop_type)])

        #note that the transition 1 --[*(]-> has aleady been treated, to transitionning to 1 won't pause any problem

        #clean should_transitionate
        circuit.append(MCXGate(2+nb_bits_position+nb_bits_loop_type,ctrl_state=n_to_ctrl_state(6,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[should_transitionate])





        #3.3.5.4) Transitions to loop_type=4

        #0 ((*)?         --[).]-> 4 ((*)+?                   INDIST-
        #4 ((*)+?        <-[*.]--                            -INGUISHABLE -> increment 3rd bit
        #0 ((*)?         --[)(]-> 4 ((*)+?                   INDIST-
        #4 ((*)+?        <-[*(]--                            -INGUISHABLE -> increment 3rd bit
        #4 ((*)+?        --[*)]-> 4 ((*)+)        bulge
        #use an ancillary qbit could_transitionate to detect the case 0 --[*)]-> 4 (the only case that  is not in the list above)
        circuit.x(could_transitionate)
        then_append(circuit,k_count,nb_bits_position,MCXGate((nb_bits_loop_type+nb_bits_third_bit_loop_type)+1,ctrl_state="1"+n_to_ctrl_state(0,nb_bits_loop_type+nb_bits_third_bit_loop_type)),[loop_type[i] for i in range(nb_bits_loop_type)]+[third_bit_loop_type[i] for i in range(nb_bits_third_bit_loop_type)]+[folding[nb_bits_bracket_dot-1],could_transitionate])

        #everyone verifying 0 == loop_type[0] and 1 == loop_type[1] transitionates (that is increment third_bit_loop_type) as far as they could tansitionate
        circuit.append(incr_position.control(2+nb_bits_position+nb_bits_loop_type+1,ctrl_state="1"+n_to_ctrl_state(0,nb_bits_loop_type)+n_to_ctrl_state(1,nb_bits_position)+"11"),[continue_scanning,continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[could_transitionate]+[third_bit_loop_type[i] for i in range(nb_bits_third_bit_loop_type)])

        #clean could_transitionate
        #no more need to control by ) == folding[k_count] since the only 0 == loop_type that stay verify it
        #TO VERIFY : it seems to me that there is a need of then_append() here, even though there is no control by folding[k_count], but I am not sure
        then_append(circuit,k_count,nb_bits_position,MCXGate(nb_bits_loop_type+nb_bits_third_bit_loop_type,ctrl_state=n_to_ctrl_state(0,nb_bits_loop_type+nb_bits_third_bit_loop_type)),[loop_type[i] for i in range(nb_bits_loop_type)]+[third_bit_loop_type[i] for i in range(nb_bits_third_bit_loop_type)]+[could_transitionate])
        circuit.x(could_transitionate)





        #3.3.5.5) Transitions to loop_type=2 and loop_type=0
        #2 (.-?          --[.)]-> 2 (.-)          hairpin
        #2 (.-?          <-[..]--
        #0 ((*)?         --[))]-> 0 ((*))         stack
        #do nothing

    #3.4) Update count[k_left]

    #if continue_searching_right_matching_left[k_left]==1
    #then update count[k_left] for the next candidate
    if 0 == k:
        #necessarily if folding[k_left]==( then continue_searching_right_matching_left[k_left]==1
        #so it is ok not controlling by continue_searching_right_matching_left here
        #( is characterised by 1 == folding[k_count][0]
        then_append(circuit,k_count,nb_bits_position,incr_position.control(2,ctrl_state="11"),[continue_scanning,folding[0                    ]]+[count[i] for i in range(nb_bits_position)])
        #) is characterised by 1 == folding[nb_bits_bracket_dot-1]
        then_append(circuit,k_count,nb_bits_position,decr_position.control(2,ctrl_state="11"),[continue_scanning,folding[nb_bits_bracket_dot-1]]+[count[i] for i in range(nb_bits_position)])
    else:
        #( is characterised by 1 == folding[k_count][0]
        then_append(circuit,k_count,nb_bits_position,incr_position.control(3,ctrl_state="111"),[continue_scanning,continue_searching_right_matching_left,folding[0                    ]]+[count[i] for i in range(nb_bits_position)])
        #) is characterised by 1 == folding[nb_bits_bracket_dot-1]
        #so there is no need controling by folding[0]
        then_append(circuit,k_count,nb_bits_position,decr_position.control(3,ctrl_state="111"),[continue_scanning,continue_searching_right_matching_left,folding[nb_bits_bracket_dot-1]]+[count[i] for i in range(nb_bits_position)])

    #if continue_searching_right_matching_left[k_left]==0, then simply increment count[k_left] so that it never equals 0 again
    #it is ok not controlling by continue_scanning here
    if 0 != k:
        #as stated previously, if 0==k then necessarily if folding[k_left]==( then continue_searching_right_matching_left[k_left]==1
        #so there is no need doing this if 0==k
        then_append(circuit,k_count,nb_bits_position,incr_position.control(ctrl_state="0"),[continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)])

    #if 1 == count[k_left]
    start_if_equal_const(circuit,count,nb_bits_position,1)


    #3.5) Update loop[k_left]

    #if 1 == continue_searching_right_matching_left[k_left] then increment loop[k_left]
    if 0==k:
        #since the value 0==loop[k_left] is known then there is no need to incr_position in order to increment
        #a CXGate() is sufficient
        then_append(circuit,[k_count[i] for i in range(nb_bits_position)]+[count[i] for i in range(nb_bits_position)],2*nb_bits_position,CCXGate(),[continue_scanning,continue_searching_right_matching_left,loop[0]])
    else:
        then_append(circuit,[k_count[i] for i in range(nb_bits_position)]+[count[i] for i in range(nb_bits_position)],2*nb_bits_position,incr_position.control(2),[continue_scanning,continue_searching_right_matching_left]+[loop[i] for i in range(nb_bits_position)])

    #end if 1 == count[k_left]
    end_if_equal_const(circuit,count,nb_bits_position,1)

    #end if k_folding == k_count
    end_if_equal(circuit,k_count,nb_bits_position,k_folding)


    #3.6) Update continue_searching_right_matching_left

    #if count[k_left] == 0
    #then there is no matching ) to find
    circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[count[i] for i in range(nb_bits_position)]+[continue_searching_right_matching_left])


    #-------------------------
    
    for k in range(1,length):
        if truncation_point <= 10 + k:
            return circuit,semantics
        circuit.append(step,[folding[i] for i in range(nb_bits_bracket_dot)]+[k_folding[i] for i in range(nb_bits_position)]+[change_folding_encoding]+[k_count[i] for i in range(nb_bits_position)]+[count[i] for i in range(nb_bits_position)]+[k_right_matching_left[i] for i in range(nb_bits_position)]+[zero,continue_searching_right_matching_left,continue_scanning]+[loop[i] for i in range(nb_bits_position)]+[loop_type[i] for i in range(nb_bits_loop_type)]+[third_bit_loop_type[i] for i in range(nb_bits_third_bit_loop_type)])
    
    
    if truncation_point == "compute matching" or truncation_point <= 10:
        return circuit,semantics
    
    #set k_count to its initial value by adding it 2**nb_bits_position - length
    for i,bit in enumerate(n_to_ctrl_state(2**nb_bits_position - length,nb_bits_position)):
        if '1'==bit:
            circuit.append(incr_gate(i+1),[k_count[j] for j in range(nb_bits_position-i-1,nb_bits_position)])

    if truncation_point == "restore k_count" or truncation_point <= 10 + length:
        return circuit,semantics    
    
    #Computed parameters
    #folding[kleft] -> continue_searching_right_matching_left[k_left] * k_right_matching_left[k_left] * loop                                 * k_count[k_left] * count[k_left]
    #(              -> 0 if a matching ) has been found               * k_right_matching_left         * number of . (at level 1) in the loop * k_left          * unspecified
    #(              -> 1 if a matching ) has not been found           * !=k_left                      * unspecified                          * k_left          * unspecified
    #. or )         -> 0                                              * k_left                        * 0                                    * k_left          * unspecified
    
    
    #4) Aggregate results
    
    #if each ( of the folding has a matching ) and if an extra ( at the beginning does not have a matching )
    #that is, if none of the conditions of incorrect_folding is fulfilled
    #then it is a correct_folding
    circuit.add_register(incorrect_folding)
    
    #checking if there is an exceeding )
    #there is if 0 == continue_searching_right_matching_left[k_left=0]
    #since the value 0==incorrect_folding is known then there is no need to add in order to increment
    #a CXGate() is sufficient
    circuit.append(MCXGate(nb_bits_position+1,ctrl_state="0"+n_to_ctrl_state(0,nb_bits_position)),[k_count[i] for i in range(nb_bits_position)]+[continue_searching_right_matching_left,incorrect_folding[0]])
    
    if truncation_point == "check incorrect )" or truncation_point <= 11 + length:
        return circuit,semantics
    
    #sum all the incorrect flags continue_searching_right_matching_left (1 per position) into incorrect_folding
    #the sum cannot overflow since if the flag continue_searching_right_matching_left[k_left=0] == 0 (at k_left=0 incorrect beeing 0) then there is a ) somewhere
    #so continue_searching_right_matching_left[k_left=somewhere] == 0
    #so incorrect_folding is not incremented at each position
    for k in range(1,length+1):
        circuit.append(incr_position.control(nb_bits_position+1,ctrl_state="1"+n_to_ctrl_state(k,nb_bits_position)),[k_count[i] for i in range(nb_bits_position)]+[continue_searching_right_matching_left]+[incorrect_folding[i] for i in range(nb_bits_position)])
    
    if truncation_point == "check incorrect (" or truncation_point <= 12 + length:
        return circuit,semantics
    
    #it is a correct_folding if 0==incorrect_folding
    circuit.add_register(correct_folding)
    circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[incorrect_folding[i] for i in range(nb_bits_position)]+[correct_folding])
    
    return circuit,semantics

def matching_parenthesis_gate(length,truncation_point=50):
    circ,_ = matching_parenthesis_circuit(length,truncation_point)
    return circ.to_gate(label="matching")

circ,_=matching_parenthesis_circuit(2)

circ.draw()

┌───┐┌───┐     ┌───┐┌───┐                    »
               (/./h/)_0: ┤ H ├┤ H ├─────┤ X ├┤ H ├───────■────────────»
                          ├───┤└─┬─┘┌───┐└─┬─┘└─┬─┘┌───┐  │            »
               (/./h/)_1: ┤ H ├──┼──┤ H ├──┼────┼──┤ H ├──■────────────»
                          ├───┤  │  └─┬─┘  │    │  └─┬─┘  │            »
             k_folding_0: ┤ H ├──o────o────o────■────■────┼────■───────»
                          ├───┤  │    │    │    │    │    │    │       »
             k_folding_1: ┤ H ├──o────o────o────■────■────┼────┼────■──»
                          └───┘                         ┌─┴─┐  │    │  »
         change_encoding: ──────────────────────────────┤ X ├──┼────┼──»
                          ┌───┐                         └───┘┌─┴─┐  │  »
               k_count_0: ┤ H ├──────────────────────────────┤ X ├──┼──»
                          ├───┤                              └───┘┌─┴─┐»
               k_count_1: ┤ H ├───────────────────────────────────┤ X ├»
                          └───┘                                   └───┘»
                 count_0: ─────────────────────────────────────────────»
                                                                       »
                 count_1: ─────────────────────────────────────────────»
                                                                       »
               k_match_0: ─────────────────────────────────────────────»
                                                                       »
               k_match_1: ─────────────────────────────────────────────»
                                                                       »
                    zero: ─────────────────────────────────────────────»
                                                                       »
          continue_match: ─────────────────────────────────────────────»
                          ┌───┐                                        »
           continue_scan: ┤ X ├────────────────────────────────────────»
                          └───┘                                        »
                  loop_0: ─────────────────────────────────────────────»
                                                                       »
                  loop_1: ─────────────────────────────────────────────»
                                                                       »
             loop_type_0: ─────────────────────────────────────────────»
                                                                       »
             loop_type_1: ─────────────────────────────────────────────»
                                                                       »
loop_type[2]_encoding)_0: ─────────────────────────────────────────────»
                                                                       »
loop_type[2]_encoding)_1: ─────────────────────────────────────────────»
                                                                       »
             incorrect_0: ─────────────────────────────────────────────»
                                                                       »
             incorrect_1: ─────────────────────────────────────────────»
                                                                       »
                 correct: ─────────────────────────────────────────────»
                                                                       »
«                          ┌───────┐                         ┌────────┐»
«               (/./h/)_0: ┤1      ├──■──────────────────────┤0       ├»
«                          │       │  │                      │        │»
«               (/./h/)_1: ┤2      ├──┼──────────────────────┤1       ├»
«                          │       │  │                      │        │»
«             k_folding_0: ┤  mcmt ├──┼────■─────────────────┤2       ├»
«                          │       │  │    │                 │        │»
«             k_folding_1: ┤       ├──┼────┼────■────────────┤3       ├»
«          

simulations

In [ ]:
np.set_printoptions(threshold=np.inf)

farthest_truncation_points = 10
length = 2

all_truncation_points = ["any folding","any k_folding","candidate folding","encoding","init k_count","init count","init k_right_matching_left","init cst","init continue_searching_right_matching_left","init continue_scanning"]+["loop_"+str(i) for i in range(length)]+["compute matching","restore k_count","check incorrect )","check incorrect ("]


for truncation_point in range(farthest_truncation_points+1):
    backend = Aer.get_backend('statevector_simulator')
    circ,sem=matching_parenthesis_circuit(length,truncation_point)
    nb_bits = circ.width()
    job = execute(circ, backend=backend, shots=1, memory=True)
    job_result = job.result()
    res=np.asarray(job_result.get_statevector(circ))
    state = np.asarray([(sem(n_to_ctrl_state(i,nb_bits)),' ') for i,val in enumerate(res[res>10.0**(-15)])])
    print(all_truncation_points[truncation_point],circ.width())
    print(state)
    print("\n")

# Transpile for simulator
#simulator = Aer.get_backend('statevector_simulator')
#circ = transpile(circ, simulator)

# Run and get counts
#result = simulator.run(circ).result()
#result
#counts = result.get_counts(circ)
#plot_histogram(counts, title='Bell-State counts')

any folding 2
[['.' ' ']
 ['(' ' ']
 [')' ' ']
 ['h' ' ']]


any k_folding 4
[['0_kf  .' ' ']
 ['0_kf  (' ' ']
 ['0_kf  )' ' ']
 ['0_kf  h' ' ']
 ['1_kf  .' ' ']
 ['1_kf  (' ' ']
 ['1_kf  )' ' ']
 ['1_kf  h' ' ']
 ['2_kf  .' ' ']
 ['2_kf  (' ' ']
 ['2_kf  )' ' ']
 ['2_kf  h' ' ']
 ['3_kf  .' ' ']
 ['3_kf  (' ' ']
 ['3_kf  )' ' ']
 ['3_kf  h' ' ']]


candidate folding 4
[['0_kf  .' ' ']
 ['0_kf  (' ' ']
 ['0_kf  )' ' ']
 ['0_kf  h' ' ']
 ['1_kf  .' ' ']
 ['1_kf  (' ' ']
 ['1_kf  )' ' ']
 ['1_kf  h' ' ']
 ['2_kf  .' ' ']
 ['2_kf  (' ' ']]


encoding 5
[['- 0_kf  .' ' ']
 ['- 0_kf  (' ' ']
 ['- 0_kf  )' ' ']
 ['- 0_kf  h' ' ']
 ['- 1_kf  .' ' ']
 ['- 1_kf  (' ' ']
 ['- 1_kf  )' ' ']
 ['- 1_kf  h' ' ']
 ['- 2_kf  .' ' ']
 ['- 2_kf  (' ' ']]


init k_count 7
[['0_kc  - 0_kf  .' ' ']
 ['0_kc  - 0_kf  (' ' ']
 ['0_kc  - 0_kf  )' ' ']
 ['0_kc  - 0_kf  h' ' ']
 ['0_kc  - 1_kf  .' ' ']
 ['0_kc  - 1_kf  (' ' ']
 ['0_kc  - 1_kf  )' ' ']
 ['0_kc  - 1_kf  h' ' ']
 ['0_kc  - 2_kf  .' ' ']
 ['0_kc  - 